## Loading modules

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve
from scipy.optimize import brentq
from sklearn.calibration import CalibratedClassifierCV
from scipy.interpolate import interp1d
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

## Importing the CSV

Using google colab, importing CSV

In [1]:
## Using google colab

# from google.colab import drive
# drive.mount('/content/drive')

# train_data = pd.read_csv('/content/drive/MyDrive/train.csv')
# dev_data = pd.read_csv('/content/drive/MyDrive/dev.csv')
# eval_data = pd.read_csv('/content/drive/MyDrive/eval.csv')

# #checking for Duplicates
# unique_row_count_train = len(train_data.drop_duplicates())
# unique_row_count_val = len(dev_data.drop_duplicates())
# unique_row_count_test = len(eval_data.drop_duplicates())

# print("Count of unique rows:", unique_row_count_train)
# print("Count of unique rows:", unique_row_count_val)
# print("Count of unique rows:", unique_row_count_test)

# # Actual length of Data frame
# row_count_train = len(train_data)
# row_count_val = len(dev_data)
# row_count_test = len(eval_data)

# print('Row Count',row_count_train)
# print('Row Count',row_count_val)
# print('Row Count',row_count_test)

# X_train = train_data.iloc[:, 2:].values
# y_train = train_data['label'].values

# X_val = dev_data.iloc[:, 2:].values
# y_val = dev_data['label'].values

# X_test = eval_data.iloc[:, 2:].values
# y_test = eval_data['label'].values

# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_val_scaled = scaler.transform(X_val)
# X_test_scaled = scaler.transform(X_test)

Creating DataFrames with features

In [ ]:
####### Creating DataFrames with features labeled from Feature_0 to Feature_1023 ######

# X_val_df = pd.DataFrame(X_val, columns=[f'{i}' for i in range(1024)])
# X_test_df = pd.DataFrame(X_test, columns=[f'{i}' for i in range(1024)])

# y_val_df = pd.DataFrame(y_val, columns=['label'])
# y_test_df = pd.DataFrame(y_test, columns=['label'])

# label_distribution_train = train_data['label'].value_counts()
# print("Distribution of data for the label attribute in train data set:")
# print(label_distribution_train)
# label_percentage_train = (label_distribution_train / len(train_data)) * 100
# print("Percentage distribution of data for the label attribute in train data set:")
# print(label_percentage_train, '\n\n\n')

# label_distribution_val = dev_data['label'].value_counts()
# print("Distribution of data for the label attribute in validation data set:")
# print(label_distribution_val)
# label_percentage_val = (label_distribution_val / len(dev_data)) * 100
# print("Percentage distribution of data for the label attribute in val data set:")
# print(label_percentage_val, '\n\n\n')

# label_distribution_test = eval_data['label'].value_counts()
# print("Distribution of data for the label attribute in test data set:")
# print(label_distribution_test)
# label_percentage_test = (label_distribution_test / len(eval_data)) * 100
# print("Percentage distribution of data for the label attribute in test data set:")
# print(label_percentage_test, '\n\n\n')

Sampling Data

In [ ]:
# # 1a. Random Oversampling
# ros = RandomOverSampler(random_state=42)
# X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

# # 1a. SMOTE Oversampling
# smote = SMOTE(random_state=42)
# X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# # 1a. ADASYN Oversampling
# adasyn = ADASYN(random_state=42)
# X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_train, y_train)

# # 1b. Random Undersampling
# rus = RandomUnderSampler(random_state=42)
# X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

## Data Visualisation

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming you have train_data, dev_data, and eval_data DataFrames
# with columns labeled from 0 to 1023 and a column named 'label'

# Function to calculate central tendency and dispersion for each feature
def calculate_stats(data):
    stats = {}
    for feature in range(1024):
        feature_name = str(feature)
        min_val = data[feature_name].min()
        max_val = data[feature_name].max()
        range_val = max_val - min_val
        quartiles = data[feature_name].quantile([0.25, 0.5, 0.75]).to_dict()
        iqr = quartiles[0.75] - quartiles[0.25]
        five_number_summary = {
            'min': min_val,
            'Q1': quartiles[0.25],
            'median': quartiles[0.5],
            'Q3': quartiles[0.75],
            'max': max_val
        }
        std_dev = data[feature_name].std()

        # Simple outlier detection using IQR
        Q1, Q3 = quartiles[0.25], quartiles[0.75]
        IQR = Q3 - Q1
        lower_bound = Q1 - (1.5 * IQR)
        upper_bound = Q3 + (1.5 * IQR)
        outliers = data[feature_name][(data[feature_name] < lower_bound) | (data[feature_name] > upper_bound)].count()

        stats[feature_name] = {
            'min': min_val,
            'max': max_val,
            'range': range_val,
            'quartiles': quartiles,
            'IQR': iqr,
            'five_number_summary': five_number_summary,
            'std_dev': std_dev,
            'outliers': outliers
        }
    return stats

# Calculate for each dataset
train_stats = calculate_stats(train_data.drop('label', axis=1))
dev_stats = calculate_stats(dev_data.drop('label', axis=1))
eval_stats = calculate_stats(eval_data.drop('label', axis=1))

# Function to plot distributions of min, max, and outliers
def plot_distributions(stats, title):
    min_values = [stats[feature]['min'] for feature in stats]
    max_values = [stats[feature]['max'] for feature in stats]
    outliers = [stats[feature]['outliers'] for feature in stats]

    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(20, 6))

    sns.histplot(min_values, ax=axes[0], kde=True)
    axes[0].set_title(f"Distribution of Min Values - {title}")
    axes[0].set_label(f"Distribution of Min Values - {title}")

    sns.histplot(max_values, ax=axes[1], kde=True)
    axes[1].set_title(f"Distribution of Max Values - {title}")

    sns.histplot(outliers, ax=axes[2], kde=True)
    axes[2].set_title(f"Distribution of Outliers - {title}")

    plt.tight_layout()
    plt.show()

# Plot distributions for each dataset
plot_distributions(train_stats, "Train Data")
plot_distributions(dev_stats, "Dev Data")
plot_distributions(eval_stats, "Eval Data")


## Neural Networks

### NN without Data Handling

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, precision_recall_curve, auc, classification_report

def plot_confusion_matrix(y_true, y_pred, model_name):
    # Convert predictions to binary (0 or 1)
    y_pred_binary = np.array([1 if p >= 0.5 else 0 for p in y_pred])
    y_true = np.array(y_true).flatten()

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred_binary)

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
                xticklabels=['Predicted 0', 'Predicted 1'],
                yticklabels=['Actual 0', 'Actual 1'])
    plt.title(f'Confusion Matrix - {model_name}')
    plt.ylabel('Actual Values')
    plt.xlabel('Predicted Values')
    plt.tight_layout()
    plt.savefig(f'Confusion_Matrix_{model_name}.png')
    plt.show()

    # Calculate and print metrics
    tn, fp, fn, tp = cm.ravel()
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    print(f"Model: {model_name}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"True Positives: {tp}, False Positives: {fp}")
    print(f"True Negatives: {tn}, False Negatives: {fn}")
    print("-" * 50)

def plot_predicted_vs_actual(y_true, y_pred, model_name):
    plt.figure(figsize=(10, 6))

    # Convert to numpy arrays if they aren't already
    y_true_array = np.array(y_true).flatten() if hasattr(y_true, 'flatten') else np.array(y_true).flatten()
    y_pred_array = np.array(y_pred).flatten() if hasattr(y_pred, 'flatten') else np.array(y_pred).flatten()

    # Get indices for sorting by true values
    indices = np.argsort(y_true_array)

    # Plot
    plt.scatter(range(len(y_true_array)), y_true_array[indices], color='blue', label='Actual', alpha=0.5, s=10)
    plt.scatter(range(len(y_pred_array)), y_pred_array[indices], color='red', label='Predicted', alpha=0.5, s=10)

    plt.title(f'Predicted vs Actual Values - {model_name}')
    plt.xlabel('Sample Index (sorted by actual value)')
    plt.ylabel('Value')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(f'predicted_vs_actual_{model_name}.png')
    plt.show()

    # Also plot a density scatter plot
    plt.figure(figsize=(8, 8))
    plt.hist2d(y_true_array, y_pred_array, bins=20, cmap='viridis')
    plt.colorbar(label='Count')
    plt.plot([0, 1], [0, 1], 'r--')
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title(f'Density Plot of Predicted vs Actual - {model_name}')
    plt.tight_layout()
    #plt.savefig(f'Density_scatter_{model_name}.png')
    plt.show()

def plot_roc_curve(y_true, y_pred, model_name):
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - {model_name}')
    plt.legend(loc="lower right")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(f'ROC_curve_{model_name}.png')
    plt.show()

    return roc_auc

def plot_precision_recall_curve(y_true, y_pred, model_name):
    # Convert to numpy arrays if they aren't already
    y_true_array = np.array(y_true).flatten() if hasattr(y_true, 'flatten') else np.array(y_true).flatten()
    y_pred_array = np.array(y_pred).flatten() if hasattr(y_pred, 'flatten') else np.array(y_pred).flatten()

    precision, recall, thresholds = precision_recall_curve(y_true_array, y_pred_array)
    pr_auc = auc(recall, precision)

    # Calculate baseline (positive rate) correctly with numpy array
    baseline = np.mean(y_true_array)

    plt.figure(figsize=(8, 6))
    plt.plot(recall, precision, color='blue', lw=2, label=f'PR curve (area = {pr_auc:.4f})')
    plt.axhline(y=baseline, color='red', linestyle='--',
                label=f'Baseline (ratio = {baseline:.4f})')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - {model_name}')
    plt.legend(loc="lower left")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(f'precision_recall_curve_{model_name}.png')
    plt.show()

    return pr_auc

def plot_prediction_distribution(y_pred, model_name):
    # plt.figure(figsize=(10, 6))
    # sns.histplot(y_pred, bins=50, kde=True)
    # plt.axvline(x=0.5, color='red', linestyle='--', label='Decision Threshold')
    # plt.title(f'Distribution of Predictions - {model_name}')
    # plt.xlabel('Predicted Probability')
    # plt.ylabel('Count')
    # plt.legend()
    # plt.grid(True, alpha=0.3)
    # plt.tight_layout()
    # plt.show()

def plot_learning_curves(train_losses, val_losses, train_accs, val_accs, model_name):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # Loss plot
    ax1.plot(train_losses, label='Training Loss')
    ax1.plot(val_losses, label='Validation Loss')
    ax1.set_title(f'{model_name} - Loss Curves')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # Accuracy plot
    ax2.plot(train_accs, label='Training Accuracy')
    ax2.plot(val_accs, label='Validation Accuracy')
    ax2.set_title(f'{model_name} - Accuracy Curves')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Accuracy')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'learning_curves_{model_name}.png')
    plt.show()

def compare_models_roc(all_y_true, all_y_pred, model_names):
    """
    Compare ROC curves of multiple models on the same plot
    """
    plt.figure(figsize=(10, 8))

    for i, (y_true, y_pred, name) in enumerate(zip(all_y_true, all_y_pred, model_names)):
        fpr, tpr, _ = roc_curve(y_true, y_pred)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=2, label=f'{name} (AUC = {roc_auc:.4f})')

    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curves Comparison')
    plt.legend(loc="lower right")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('roc_curves_comparison.png')
    plt.show()

def plot_feature_importance(model, input_dim, model_name):
    """
    Plot feature importance for single layer perceptron
    """
    if hasattr(model, 'linear'):
        # Get weights from the model
        weights = model.linear.weight.data.cpu().numpy().flatten()

        # Plot top 20 features by magnitude
        indices = np.argsort(np.abs(weights))[-20:]
        plt.figure(figsize=(12, 6))
        plt.barh(np.arange(len(indices)), weights[indices])
        plt.yticks(np.arange(len(indices)), [f'Feature {i}' for i in indices])
        plt.title(f'Top 20 Feature Importance - {model_name}')
        plt.xlabel('Weight Magnitude')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(f'feature_importance_{model_name}.png')
        plt.show()
    else:
        print(f"Feature importance visualization not applicable for {model_name}")

# Updated training function to track accuracy as well
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device='cpu'):
    model = model.to(device)

    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []

    for epoch in range(num_epochs):
        # Training
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

            # Calculate accuracy
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct / total
        train_losses.append(epoch_loss)
        train_accs.append(epoch_acc)

        # Validation
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)

                predicted = (outputs > 0.5).float()
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_epoch_loss = val_loss / len(val_loader.dataset)
        val_epoch_acc = val_correct / val_total
        val_losses.append(val_epoch_loss)
        val_accs.append(val_epoch_acc)

        print(f'Epoch {epoch+1}/{num_epochs}, '
              f'Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}, '
              f'Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}')

    return train_losses, val_losses, train_accs, val_accs

# Example of usage with all the visualization functions
# Hyperparameters
input_dim = 1024
batch_size = 64
learning_rate = 0.001
num_epochs = 20

# Check for GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Prepare data
train_loader, val_loader, test_loader = prepare_data(
    X_train, y_train, X_val, y_val, X_test, y_test, batch_size)

# Define loss function
criterion = nn.BCELoss()

# Lists to store results for comparison
all_model_names = ["Single Layer Perceptron", "Multi-Layer Perceptron", "Deep Neural Network", "Artificial Neural Network"]
all_test_y_true = []
all_test_y_pred = []
all_roc_aucs = []
all_pr_aucs = []
all_test_accs = []

# 1. Train and evaluate Single Layer Perceptron
print("\n=== Training Single Layer Perceptron ===")
slp_model = SingleLayerPerceptron(input_dim)
slp_optimizer = optim.Adam(slp_model.parameters(), lr=learning_rate)
slp_train_losses, slp_val_losses, slp_train_accs, slp_val_accs = train_model(
    slp_model, train_loader, val_loader, criterion, slp_optimizer, num_epochs, device)

# Evaluate on test set
slp_test_loss, slp_test_acc, slp_preds, slp_targets = evaluate_model(
    slp_model, test_loader, criterion, device)
all_test_y_true.append(slp_targets)
all_test_y_pred.append(slp_preds)
all_test_accs.append(slp_test_acc)

# Visualizations for Single Layer Perceptron
model_name = "Single Layer Perceptron"
plot_learning_curves(slp_train_losses, slp_val_losses, slp_train_accs, slp_val_accs, model_name)
plot_confusion_matrix(slp_targets, slp_preds, model_name)
plot_predicted_vs_actual(slp_targets, slp_preds, model_name)
slp_roc_auc = plot_roc_curve(slp_targets, slp_preds, model_name)
slp_pr_auc = plot_precision_recall_curve(slp_targets, slp_preds, model_name)
plot_prediction_distribution(slp_preds, model_name)
plot_feature_importance(slp_model, input_dim, model_name)
all_roc_aucs.append(slp_roc_auc)
all_pr_aucs.append(slp_pr_auc)

# 2. Train and evaluate Multi-Layer Perceptron
print("\n=== Training Multi-Layer Perceptron ===")
mlp_model = MultiLayerPerceptron(input_dim)
mlp_optimizer = optim.Adam(mlp_model.parameters(), lr=learning_rate)
mlp_train_losses, mlp_val_losses, mlp_train_accs, mlp_val_accs = train_model(
    mlp_model, train_loader, val_loader, criterion, mlp_optimizer, num_epochs, device)

# Evaluate on test set
mlp_test_loss, mlp_test_acc, mlp_preds, mlp_targets = evaluate_model(
    mlp_model, test_loader, criterion, device)
all_test_y_true.append(mlp_targets)
all_test_y_pred.append(mlp_preds)
all_test_accs.append(mlp_test_acc)

# Visualizations for Multi-Layer Perceptron
model_name = "Multi-Layer Perceptron"
plot_learning_curves(mlp_train_losses, mlp_val_losses, mlp_train_accs, mlp_val_accs, model_name)
plot_confusion_matrix(mlp_targets, mlp_preds, model_name)
plot_predicted_vs_actual(mlp_targets, mlp_preds, model_name)
mlp_roc_auc = plot_roc_curve(mlp_targets, mlp_preds, model_name)
mlp_pr_auc = plot_precision_recall_curve(mlp_targets, mlp_preds, model_name)
plot_prediction_distribution(mlp_preds, model_name)
all_roc_aucs.append(mlp_roc_auc)
all_pr_aucs.append(mlp_pr_auc)

# 3. Train and evaluate Deep Neural Network
print("\n=== Training Deep Neural Network ===")
dnn_model = DeepNeuralNetwork(input_dim)
dnn_optimizer = optim.Adam(dnn_model.parameters(), lr=learning_rate)
dnn_train_losses, dnn_val_losses, dnn_train_accs, dnn_val_accs = train_model(
    dnn_model, train_loader, val_loader, criterion, dnn_optimizer, num_epochs, device)

# Evaluate on test set
dnn_test_loss, dnn_test_acc, dnn_preds, dnn_targets = evaluate_model(
    dnn_model, test_loader, criterion, device)
all_test_y_true.append(dnn_targets)
all_test_y_pred.append(dnn_preds)
all_test_accs.append(dnn_test_acc)

# Visualizations for Deep Neural Network
model_name = "Deep Neural Network"
plot_learning_curves(dnn_train_losses, dnn_val_losses, dnn_train_accs, dnn_val_accs, model_name)
plot_confusion_matrix(dnn_targets, dnn_preds, model_name)
plot_predicted_vs_actual(dnn_targets, dnn_preds, model_name)
dnn_roc_auc = plot_roc_curve(dnn_targets, dnn_preds, model_name)
dnn_pr_auc = plot_precision_recall_curve(dnn_targets, dnn_preds, model_name)
plot_prediction_distribution(dnn_preds, model_name)
all_roc_aucs.append(dnn_roc_auc)
all_pr_aucs.append(dnn_pr_auc)

# 4. Train and evaluate Artificial Neural Network
print("\n=== Training Artificial Neural Network ===")
ann_model = ArtificialNeuralNetwork(input_dim)
ann_optimizer = optim.Adam(ann_model.parameters(), lr=learning_rate, weight_decay=1e-5)
ann_train_losses, ann_val_losses, ann_train_accs, ann_val_accs = train_model(
    ann_model, train_loader, val_loader, criterion, ann_optimizer, num_epochs, device)

# Evaluate on test set
ann_test_loss, ann_test_acc, ann_preds, ann_targets = evaluate_model(
    ann_model, test_loader, criterion, device)
all_test_y_true.append(ann_targets)
all_test_y_pred.append(ann_preds)
all_test_accs.append(ann_test_acc)

# Visualizations for Artificial Neural Network
model_name = "Artificial Neural Network"
plot_learning_curves(ann_train_losses, ann_val_losses, ann_train_accs, ann_val_accs, model_name)
plot_confusion_matrix(ann_targets, ann_preds, model_name)
plot_predicted_vs_actual(ann_targets, ann_preds, model_name)
ann_roc_auc = plot_roc_curve(ann_targets, ann_preds, model_name)
ann_pr_auc = plot_precision_recall_curve(ann_targets, ann_preds, model_name)
plot_prediction_distribution(ann_preds, model_name)
all_roc_aucs.append(ann_roc_auc)
all_pr_aucs.append(ann_pr_auc)

# Compare all models
print("\n=== Model Comparison ===")
for name, acc, roc, pr in zip(all_model_names, all_test_accs, all_roc_aucs, all_pr_aucs):
    print(f"{name}: Test Acc={acc:.4f}, ROC AUC={roc:.4f}, PR AUC={pr:.4f}")

# Plot comparative ROC curves
compare_models_roc(all_test_y_true, all_test_y_pred, all_model_names)

# Plot accuracy comparison
plt.figure(figsize=(10, 6))
plt.bar(all_model_names, all_test_accs)
plt.title('Model Accuracy Comparison')
plt.xlabel('Model')
plt.ylabel('Test Accuracy')
plt.ylim(0, 1)
plt.grid(axis='y')
plt.tight_layout()
plt.savefig('accuracy_comparison_NN_withoutDataHandling.png')
plt.show()

# Plot AUC comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# ROC AUC comparison
ax1.bar(all_model_names, all_roc_aucs)
ax1.set_title('ROC AUC Comparison')
ax1.set_xlabel('Model')
ax1.set_ylabel('ROC AUC')
ax1.set_ylim(0, 1)
ax1.grid(axis='y')

# PR AUC comparison
ax2.bar(all_model_names, all_pr_aucs)
ax2.set_title('Precision-Recall AUC Comparison')
ax2.set_xlabel('Model')
ax2.set_ylabel('PR AUC')
ax2.set_ylim(0, 1)
ax2.grid(axis='y')

plt.tight_layout()
plt.savefig('auc_comparison_NN_withoutDataHandling.png')
plt.show()


### NN with Data Handling


Sampling the DataSet and Using PCA

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc

# For sampling strategies
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# ----------------------------
# ASSUMPTIONS:
# Your datasets should already be defined and preprocessed
# X_train, y_train, X_val, y_val, X_test, y_test
# ----------------------------

# Define class weights (you can adjust based on your imbalance needs)
class_weights = {0: 0.1, 1: 0.9}

# --- Function to Create Neural Network Models ---
def create_models(input_dim):
    models = {}

    # 1. Single-Layer Neural Network
    model_single = Sequential([
        Dense(1, activation='sigmoid', input_shape=(input_dim,))
    ])
    model_single.compile(optimizer=Adam(0.001),
                         loss='binary_crossentropy',
                         metrics=['accuracy'])
    models['Single-Layer'] = model_single

    # 2. Multi-Layer Neural Network (2 layers)
    model_multi = Sequential([
        Dense(512, activation='relu', input_shape=(input_dim,)),
        Dense(1, activation='sigmoid')
    ])
    model_multi.compile(optimizer=Adam(0.001),
                        loss='binary_crossentropy',
                        metrics=['accuracy'])
    models['Multi-Layer'] = model_multi

    # 3. Deep Neural Network (4 layers)
    model_deep = Sequential([
        Dense(512, activation='relu', input_shape=(input_dim,)),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model_deep.compile(optimizer=Adam(0.001),
                        loss='binary_crossentropy',
                        metrics=['accuracy'])
    models['Deep'] = model_deep

    # 4. Regularized ANN (with dropout and L2)
    model_reg = Sequential([
        Dense(512, activation='relu', kernel_regularizer=l2(0.001), input_shape=(input_dim,)),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model_reg.compile(optimizer=Adam(0.001),
                      loss='binary_crossentropy',
                      metrics=['accuracy'])
    models['Regularized'] = model_reg

    return models

# --- Plotting helper functions ---
def plot_history(history, title):
    plt.figure(figsize=(12, 5))

    # Plot Loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title(f'{title} - Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Plot Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.title(f'{title} - Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

def plot_confusion_matrix(cm, title):
    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(title)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

def plot_roc_curve(fpr, tpr, roc_auc, title):
    plt.figure(figsize=(6,4))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC (AUC = {roc_auc:.2f})')
    plt.plot([0,1], [0,1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()

# --- Define Sampling Strategies and PCA Dimensions ---
# For oversampling, we use SMOTE; for undersampling, use RandomUnderSampler.
sampling_strategies = {
    'SMOTE': SMOTE(random_state=42),
    'Undersampling': RandomUnderSampler(random_state=42)
}

pca_dimensions = [50, 100, 200]

# ----------------------------
# Loop over each sampling strategy and PCA dimension
for sample_method, sampler in sampling_strategies.items():
    for n_components in pca_dimensions:
        print(f"\n=== Running {sample_method} with PCA (n_components = {n_components}) ===\n")

        # 1. Apply sampling only to the training set
        X_train_res, y_train_res = sampler.fit_resample(X_train, y_train)
        print(f"Resampled training set shape: {X_train_res.shape}, Class distribution: {np.bincount(y_train_res)}")

        # 2. Apply PCA on the resampled training data and transform validation and test sets
        pca = PCA(n_components=n_components, random_state=42)
        X_train_pca = pca.fit_transform(X_train_res)
        X_val_pca   = pca.transform(X_val)
        X_test_pca  = pca.transform(X_test)
        print(f"PCA-transformed train shape: {X_train_pca.shape}")

        # 3. Create neural network models adjusted for the new input dimension (n_components)
        models = create_models(input_dim=n_components)
        histories = {}  # To store training history for each model

        # 4. Train each model on the PCA-transformed, resampled training data
        for model_name, model in models.items():
            print(f"Training model: {model_name}")
            history = model.fit(
                X_train_pca, y_train_res,
                validation_data=(X_val_pca, y_val),
                epochs=30,
                batch_size=32,
                class_weight=class_weights,
                verbose=0
            )
            histories[model_name] = history
            # Plot training history for this model
            plot_history(history, f"{model_name} ({sample_method}, PCA={n_components})")

        # 5. Combined Validation Accuracy Plot for all models in this experiment
        plt.figure(figsize=(14, 6))
        for model_name, history in histories.items():
            plt.plot(history.history['val_accuracy'], label=f'{model_name}')
        plt.title(f'Model Comparison - Val Accuracy ({sample_method}, PCA={n_components})')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.show()

        # 6. Evaluate each model on the test data (transformed by PCA)
        for model_name, model in models.items():
            print(f"Evaluating model: {model_name} ({sample_method}, PCA={n_components})")
            # Predict probabilities and then binarize predictions using a threshold of 0.5
            y_pred_prob = model.predict(X_test_pca)
            y_pred = (y_pred_prob > 0.5).astype(int)

            # Generate and plot Confusion Matrix
            cm = confusion_matrix(y_test, y_pred)
            plot_confusion_matrix(cm, f'{model_name} - Confusion Matrix ({sample_method}, PCA={n_components})')

            # Print Classification Report
            print(f'\n{model_name} - Classification Report ({sample_method}, PCA={n_components}):')
            print(classification_report(y_test, y_pred))

            # Plot ROC Curve
            fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
            roc_auc = auc(fpr, tpr)
            plot_roc_curve(fpr, tpr, roc_auc, f'{model_name} - ROC Curve ({sample_method}, PCA={n_components})')


## Adam Optimisor & Cross Entropy Loss Function for NN

Iterating through multiple models for Hyper-parameter Tuning & Threshold tuning

Changing the class weights and checking for the results...


These are multiple codes.....

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import pandas as pd

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Working with existing variables: X_train, y_train, X_val, y_val, X_test, y_test
# Assuming these are already defined with binary classes (0 and 1)
# and there are 1024 features

# Get input dimension from the data
input_dim = 1024  # As specified
n_classes = 2  # Binary classification with classes 0 and 1

# Store original labels for metric calculation
y_train_orig = y_train.copy()
y_val_orig = y_val.copy()
y_test_orig = y_test.copy()

# Calculate class weights for weighted cross-entropy
def calculate_class_weights(y):
    class_counts = np.bincount(y.astype(int))
    total = len(y)

    # Inverse frequency weighting
    class_weights = total / (len(class_counts) * class_counts)

    # Convert to dictionary format for Keras
    weights_dict = {i: weight for i, weight in enumerate(class_weights)}
    return weights_dict, class_weights

weights_dict, class_weights_array = calculate_class_weights(y_train)
print("Class weights:", weights_dict)

# Define a custom weighted binary cross-entropy loss function
def weighted_binary_crossentropy(class_weights):
    def loss(y_true, y_pred):
        # Standard binary crossentropy calculation
        bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)

        # Create weight vector based on class values in y_true
        weights = y_true * class_weights[1] + (1 - y_true) * class_weights[0]

        # Apply weights to the bce loss
        weighted_bce = weights * bce

        return tf.reduce_mean(weighted_bce)

    return loss

# Define a function to create and compile a neural network
def create_neural_network(model_type, input_dim, class_weights_array):
    model = Sequential()

    if model_type == "Single Layer":
        model.add(Dense(units=1, input_dim=input_dim, activation='sigmoid'))

    elif model_type == "Multi-layer":
        model.add(Dense(units=64, input_dim=input_dim, activation='relu'))
        model.add(Dense(units=1, activation='sigmoid'))

    elif model_type == "Deep NN":
        model.add(Dense(units=128, input_dim=input_dim, activation='relu'))
        model.add(Dense(units=64, activation='relu'))
        model.add(Dense(units=32, activation='relu'))
        model.add(Dense(units=1, activation='sigmoid'))

    elif model_type == "Artificial NN":
        model.add(Dense(units=256, input_dim=input_dim, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(units=128, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(units=64, activation='relu'))
        model.add(Dense(units=1, activation='sigmoid'))

    # Custom weighted binary cross-entropy loss function
    weighted_loss = weighted_binary_crossentropy(class_weights_array)

    # Compile the model with our custom weighted loss
    model.compile(optimizer=Adam(learning_rate=0.001),
                 loss=weighted_loss,
                 metrics=['accuracy'])

    return model

# Function for training and evaluation
def train_and_evaluate(model_type, X_train, y_train, X_val, y_val, X_test, y_test, class_weights_array):
    # Create model
    model = create_neural_network(model_type, input_dim, class_weights_array)

    # Set up early stopping
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )

    # Train the model
    history = model.fit(
        X_train, y_train,
        epochs=20,
        batch_size=32,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping],
        verbose=1
    )

    # Evaluate on test set
    y_pred_prob = model.predict(X_test)
    y_pred = (y_pred_prob > 0.5).astype(int).flatten()

    # Calculate metrics
    accuracy = accuracy_score(y_test_orig, y_pred)
    precision = precision_score(y_test_orig, y_pred)
    recall = recall_score(y_test_orig, y_pred)
    f1 = f1_score(y_test_orig, y_pred)

    # Create confusion matrix
    cm = confusion_matrix(y_test_orig, y_pred)

    metrics = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

    return model, history, metrics, cm

# List of models to train
model_types = ["Single Layer", "Multi-layer", "Deep NN", "Artificial NN"]
model_results = {}

# Train each model
for model_type in model_types:
    print(f"\nTraining {model_type} Neural Network...")
    model, history, metrics, cm = train_and_evaluate(
        model_type, X_train, y_train, X_val, y_val, X_test, y_test, class_weights_array
    )
    model_results[model_type] = {
        'model': model,
        'history': history,
        'metrics': metrics,
        'confusion_matrix': cm
    }
    print(f"{model_type} Metrics:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.4f}")

# Plotting functions
def plot_training_history(histories, model_types):
    plt.figure(figsize=(12, 10))

    # Plot training & validation accuracy
    plt.subplot(2, 1, 1)
    for model_type in model_types:
        history = histories[model_type]['history']
        plt.plot(history.history['accuracy'], label=f'{model_type} Training')
        plt.plot(history.history['val_accuracy'], label=f'{model_type} Validation')

    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(loc='lower right')

    # Plot training & validation loss
    plt.subplot(2, 1, 2)
    for model_type in model_types:
        history = histories[model_type]['history']
        plt.plot(history.history['loss'], label=f'{model_type} Training')
        plt.plot(history.history['val_loss'], label=f'{model_type} Validation')

    plt.title('Model Loss (Weighted Cross-Entropy)')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(loc='upper right')

    plt.tight_layout()
    plt.savefig('training_history_comparison.png')
    plt.show()

def plot_confusion_matrices(model_results, model_types):
    n = len(model_types)
    fig, axes = plt.subplots(1, n, figsize=(n*5, 5))

    if n == 1:
        axes = [axes]  # Handle the case of a single plot

    for i, model_type in enumerate(model_types):
        cm = model_results[model_type]['confusion_matrix']
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i])
        axes[i].set_title(f'{model_type} Confusion Matrix')
        axes[i].set_xlabel('Predicted labels')
        axes[i].set_ylabel('True labels')

    plt.tight_layout()
    plt.savefig('confusion_matrices.png')
    plt.show()

def plot_metrics_comparison(model_results, model_types):
    metrics = ['accuracy', 'precision', 'recall', 'f1']

    # Prepare data for plotting
    df_metrics = pd.DataFrame({
        metric: [model_results[model_type]['metrics'][metric] for model_type in model_types]
        for metric in metrics
    }, index=model_types)

    # Plot
    plt.figure(figsize=(10, 6))
    df_metrics.plot(kind='bar', rot=0)
    plt.title('Performance Comparison Across Models')
    plt.ylabel('Score')
    plt.ylim(0, 1)
    plt.legend(title='Metric')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig('metrics_comparison.png')
    plt.show()

def plot_individual_model_metrics(model_results):
    # Plot separate metrics for each model
    for model_type in model_results:
        plt.figure(figsize=(8, 6))
        metrics = model_results[model_type]['metrics']
        plt.bar(metrics.keys(), metrics.values())
        plt.title(f'{model_type} Performance Metrics')
        plt.ylabel('Score')
        plt.ylim(0, 1)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.savefig(f'{model_type.replace(" ", "_").lower()}_metrics.png')
        plt.show()

# Generate plots
plot_training_history(model_results, model_types)
plot_confusion_matrices(model_results, model_types)
plot_metrics_comparison(model_results, model_types)
plot_individual_model_metrics(model_results)

# Summary of results
print("\n===== RESULTS SUMMARY =====")
for model_type in model_types:
    metrics = model_results[model_type]['metrics']
    print(f"\n{model_type} Neural Network:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.4f}")

# Create a recommendation based on the results
best_model = max(model_types, key=lambda x: model_results[x]['metrics']['f1'])
print(f"\nRecommended model: {best_model} (based on F1 score)")
print(f"F1 Score: {model_results[best_model]['metrics']['f1']:.4f}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

# Define class weights (adjust based on class imbalance)
class_weights = {0: 0.1, 1: 0.9}

# Initialize history storage
histories = {}

# 1. Single-Layer Neural Network
model_single = Sequential([
    Dense(1, activation='sigmoid', input_shape=(1024,))
])
model_single.compile(optimizer=Adam(0.001),
                    loss='binary_crossentropy',
                    metrics=['accuracy'])
histories['Single-Layer'] = model_single.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=32,
    class_weight=class_weights,
    verbose=0
)

# 2. Multi-Layer Neural Network (2 layers)
model_multi = Sequential([
    Dense(512, activation='relu', input_shape=(1024,)),
    Dense(1, activation='sigmoid')
])
model_multi.compile(optimizer=Adam(0.001),
                   loss='binary_crossentropy',
                   metrics=['accuracy'])
histories['Multi-Layer'] = model_multi.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=32,
    class_weight=class_weights,
    verbose=0
)

# 3. Deep Neural Network (4 layers)
model_deep = Sequential([
    Dense(512, activation='relu', input_shape=(1024,)),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])
model_deep.compile(optimizer=Adam(0.001),
                 loss='binary_crossentropy',
                 metrics=['accuracy'])
histories['Deep'] = model_deep.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=32,
    class_weight=class_weights,
    verbose=0
)

# 4. Regularized ANN (with dropout and L2)
model_reg = Sequential([
    Dense(512, activation='relu', kernel_regularizer=l2(0.001), input_shape=(1024,)),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])
model_reg.compile(optimizer=Adam(0.001),
                loss='binary_crossentropy',
                metrics=['accuracy'])
histories['Regularized'] = model_reg.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=32,
    class_weight=class_weights,
    verbose=0
)

# Individual model plots
def plot_history(history, title):
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title(f'{title} - Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.title(f'{title} - Accuracy')
    plt.legend()
    plt.show()

for name, history in histories.items():
    plot_history(history, name)

# Comparison plot
plt.figure(figsize=(14, 6))
for model_name, history in histories.items():
    plt.plot(history.history['val_accuracy'], label=f'{model_name} Val Accuracy')
plt.title('Model Comparison - Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()



import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc

def evaluate_model(model, X_test, y_test):
    # Predict probabilities and classes
    y_probs = model.predict(X_test, verbose=0)
    y_pred = (y_probs > 0.5).astype(int)

    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(6,5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'{model.name} Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

    # ROC Curve
    fpr, tpr, _ = roc_curve(y_test, y_probs)
    roc_auc = auc(fpr, tpr)
    plt.figure()
    plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.2f}')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.title(f'{model.name} ROC Curve')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='lower right')
    plt.show()

    # Classification Report
    print(f"Classification Report for {model.name}:")
    print(classification_report(y_test, y_pred, target_names=['Class 0', 'Class 1']))

# Load saved models and evaluate
models = {
    'Single-Layer': model_single,
    'Multi-Layer': model_multi,
    'Deep': model_deep,
    'Regularized': model_reg
}

for name, model in models.items():
    model._name = name  # Set model name for labeling
    evaluate_model(model, X_test, y_test)


## SVM (Yash)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import svm

# Assume X and y are your 2D dataset and labels
# For demonstration, let's generate a synthetic dataset
#from sklearn.datasets import make_classification
#X, y = make_classification(n_samples=200, n_features=2, n_redundant=0, n_clusters_per_class=1, weights=[0.9], flip_y=0, random_state=4)

# Apply SVM with a linear kernel for simplicity
svm_model = svm.SVC(kernel='linear')

# Train the model
svm_model.fit(X_train_pca, y_train)

# Create a grid of points to visualize the decision boundary
h = .02  # step size in the mesh
x_min, x_max = X_train_pca[:, 0].min() - 1, X_train_pca[:, 0].max() + 1
y_min, y_max = X_train_pca[:, 1].min() - 1, X_train_pca[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

# Predict probabilities or decision function values for the grid
Z = svm_model.decision_function(np.c_[xx.ravel(), yy.ravel()])

# Reshape Z to match the grid dimensions
Z = Z.reshape(xx.shape)

# Plotting
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot data points
ax.scatter(X_train_pca[y_train == 0, 0], X_train_pca[y_train == 0, 1], np.zeros(len(X_train_pca[y_train == 0])), color='orange', label='Class 0')
ax.scatter(X_train_pca[y_train == 1, 0], X_train_pca[y_train == 1, 1], np.zeros(len(X_train_pca[y_train == 1])), color='blue', label='Class 1')

# Plot the decision boundary as a surface
ax.plot_surface(xx, yy, Z, alpha=0.5, cmap='coolwarm')

ax.set_title('SVM Decision Boundary in 3D')
ax.set_xlabel('Feature 1')
ax.set_ylabel('Feature 2')
ax.set_zlabel('Decision Function Value')
ax.legend()
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, precision_recall_curve
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

# Set plotting style and figure size for better visualization
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

def evaluate_model(model, X, y, set_name):
    y_pred = model.predict(X)

    # Calculate metrics (setting zero_division=0 to handle cases where precision/recall might be undefined)
    accuracy = accuracy_score(y, y_pred)
    try:
        precision = precision_score(y, y_pred, zero_division=0)
        recall = recall_score(y, y_pred, zero_division=0)
        f1 = f1_score(y, y_pred, zero_division=0)
    except:
        precision, recall, f1 = 0, 0, 0

    print(f"\n--- {set_name} Metrics ---")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    return y_pred, accuracy, precision, recall, f1

def plot_confusion_matrix(y_true, y_pred, set_name, save=True):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title(f'Confusion Matrix - {set_name}')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    if save:
        plt.savefig(f'confusion_matrix_{set_name.lower().replace(" ", "_")}.png', dpi=300, bbox_inches='tight')
    plt.show()

    # Calculate and print additional metrics from confusion matrix
    if cm.size == 4:  # Binary classification
        tn, fp, fn, tp = cm.ravel()
        print(f"True Negatives: {tn}")
        print(f"False Positives: {fp}")
        print(f"False Negatives: {fn}")
        print(f"True Positives: {tp}")

def plot_roc_curve(model, X, y, set_name, save=True):
    try:
        # Get probability estimates
        y_prob = model.decision_function(X)

        # Calculate ROC curve and AUC
        fpr, tpr, _ = roc_curve(y, y_prob)
        roc_auc = auc(fpr, tpr)

        # Plot ROC curve
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'Receiver Operating Characteristic (ROC) - {set_name}')
        plt.legend(loc="lower right")
        if save:
            plt.savefig(f'roc_curve_{set_name.lower().replace(" ", "_")}.png', dpi=300, bbox_inches='tight')
        plt.show()

        return roc_auc
    except:
        print(f"Could not generate ROC curve for {set_name} - the model might not support decision_function or the data is too imbalanced")
        return None

def plot_precision_recall_curve(model, X, y, set_name, save=True):
    try:
        # Get probability estimates
        y_prob = model.decision_function(X)

        # Calculate precision-recall curve
        precision, recall, _ = precision_recall_curve(y, y_prob)

        # Plot precision-recall curve
        plt.figure(figsize=(8, 6))
        plt.plot(recall, precision, color='green', lw=2)
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.ylim([0.0, 1.05])
        plt.xlim([0.0, 1.0])
        plt.title(f'Precision-Recall Curve - {set_name}')
        if save:
            plt.savefig(f'precision_recall_curve_{set_name.lower().replace(" ", "_")}.png', dpi=300, bbox_inches='tight')
        plt.show()
    except:
        print(f"Could not generate Precision-Recall curve for {set_name} - the model might not support decision_function or the data is too imbalanced")

def plot_decision_boundary(model, X, y, set_name, save=True):
    if X.shape[1] != 2:
        print(f"Cannot plot decision boundary for {set_name} - requires exactly 2 features")
        return

    # Create mesh grid
    h = 0.02  # step size in the mesh
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    # Get predictions on the mesh grid
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    # Plot the decision boundary and scatter points
    plt.figure(figsize=(10, 8))
    plt.contourf(xx, yy, Z, alpha=0.8, cmap=plt.cm.RdBu)
    plt.scatter(X[:, 0], X[:, 1], c=y, edgecolors='k', cmap=plt.cm.RdBu)
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.title(f'Decision Boundary - {set_name}')
    if save:
        plt.savefig(f'decision_boundary_{set_name.lower().replace(" ", "_")}.png', dpi=300, bbox_inches='tight')
    plt.show()

def run_grid_search_for_kernel(X_train, y_train, X_val, y_val, kernel_type):
    print(f"\nPerforming grid search for {kernel_type} kernel...")

    # Define parameter grid based on kernel type
    if kernel_type == 'linear':
        param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
    elif kernel_type == 'poly':
        param_grid = {'C': [0.1, 1, 10, 100], 'degree': [2, 3, 4, 5], 'gamma': ['scale', 'auto', 0.01, 0.1, 1]}
    else:  # rbf
        param_grid = {'C': [0.1, 1, 10, 100], 'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1]}

    # Initialize SVM model with the specific kernel
    base_model = SVC(kernel=kernel_type, probability=True, class_weight='balanced')

    # Initialize GridSearchCV
    grid = GridSearchCV(base_model,
                      param_grid,
                      cv=5,
                      scoring='f1',
                      verbose=1,
                      n_jobs=-1)

    # Fit GridSearchCV
    grid.fit(X_train, y_train)

    # Display best parameters and score
    print(f"\nBest parameters for {kernel_type} kernel: {grid.best_params_}")
    print(f"Best cross-validation score: {grid.best_score_:.4f}")

    # Evaluate best model on validation set
    best_model = grid.best_estimator_
    y_val_pred = best_model.predict(X_val)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    val_precision = precision_score(y_val, y_val_pred, zero_division=0)
    val_recall = recall_score(y_val, y_val_pred, zero_division=0)
    val_f1 = f1_score(y_val, y_val_pred, zero_division=0)

    print(f"\nValidation set performance:")
    print(f"Accuracy: {val_accuracy:.4f}")
    print(f"Precision: {val_precision:.4f}")
    print(f"Recall: {val_recall:.4f}")
    print(f"F1 Score: {val_f1:.4f}")

    return best_model, grid.best_params_, val_f1

def plot_hyperparameter_comparison(X_train, y_train, X_val, y_val, kernel_type, best_params):
    plt.figure(figsize=(15, 10))

    # Define hyperparameters to test based on kernel type
    if kernel_type == 'linear':
        C_values = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
        train_scores = []
        val_scores = []

        for C in C_values:
            model = SVC(kernel='linear', C=C, class_weight='balanced')
            model.fit(X_train, y_train)

            train_pred = model.predict(X_train)
            val_pred = model.predict(X_val)

            train_f1 = f1_score(y_train, train_pred, zero_division=0)
            val_f1 = f1_score(y_val, val_pred, zero_division=0)

            train_scores.append(train_f1)
            val_scores.append(val_f1)

        plt.plot(C_values, train_scores, 'o-', label='Training F1 Score')
        plt.plot(C_values, val_scores, 's-', label='Validation F1 Score')
        plt.axvline(x=best_params['C'], color='r', linestyle='--',
                    label=f'Best C: {best_params["C"]}')
        plt.xscale('log')
        plt.xlabel('C (Regularization parameter)')
        plt.ylabel('F1 Score')
        plt.title(f'Impact of C on Linear SVM Performance')
        plt.legend()
        plt.grid(True)
        plt.savefig('linear_svm_hyperparameter_comparison.png', dpi=300, bbox_inches='tight')
        plt.show()

    elif kernel_type == 'poly':
        C_values = [0.1, 1, 10, 100]
        degrees = [2, 3, 4, 5]

        fig, ax = plt.subplots(1, 2, figsize=(15, 6))

        # Test different C values with selected optimal degree
        best_degree = best_params['degree']
        train_scores_c = []
        val_scores_c = []
        for C in C_values:
            model = SVC(kernel='poly', C=C, degree=best_degree, gamma=best_params['gamma'],
                       class_weight='balanced')
            model.fit(X_train, y_train)

            train_pred = model.predict(X_train)
            val_pred = model.predict(X_val)

            train_f1 = f1_score(y_train, train_pred, zero_division=0)
            val_f1 = f1_score(y_val, val_pred, zero_division=0)

            train_scores_c.append(train_f1)
            val_scores_c.append(val_f1)

        ax[0].plot(C_values, train_scores_c, 'o-', label='Training F1 Score')
        ax[0].plot(C_values, val_scores_c, 's-', label='Validation F1 Score')
        ax[0].axvline(x=best_params['C'], color='r', linestyle='--',
                     label=f'Best C: {best_params["C"]}')
        ax[0].set_xscale('log')
        ax[0].set_xlabel('C (Regularization parameter)')
        ax[0].set_ylabel('F1 Score')
        ax[0].set_title(f'Impact of C on Polynomial SVM (degree={best_degree})')
        ax[0].legend()
        ax[0].grid(True)

        # Test different degree values with selected optimal C
        best_C = best_params['C']
        train_scores_d = []
        val_scores_d = []
        for degree in degrees:
            model = SVC(kernel='poly', C=best_C, degree=degree, gamma=best_params['gamma'],
                       class_weight='balanced')
            model.fit(X_train, y_train)

            train_pred = model.predict(X_train)
            val_pred = model.predict(X_val)

            train_f1 = f1_score(y_train, train_pred, zero_division=0)
            val_f1 = f1_score(y_val, val_pred, zero_division=0)

            train_scores_d.append(train_f1)
            val_scores_d.append(val_f1)

        ax[1].plot(degrees, train_scores_d, 'o-', label='Training F1 Score')
        ax[1].plot(degrees, val_scores_d, 's-', label='Validation F1 Score')
        ax[1].axvline(x=best_params['degree'], color='r', linestyle='--',
                     label=f'Best degree: {best_params["degree"]}')
        ax[1].set_xlabel('Polynomial Degree')
        ax[1].set_ylabel('F1 Score')
        ax[1].set_title(f'Impact of Degree on Polynomial SVM (C={best_C})')
        ax[1].legend()
        ax[1].grid(True)

        plt.tight_layout()
        plt.savefig('poly_svm_hyperparameter_comparison.png', dpi=300, bbox_inches='tight')
        plt.show()

    elif kernel_type == 'rbf':
        C_values = [0.1, 1, 10, 100]
        if 'scale' in best_params['gamma'] or 'auto' in best_params['gamma']:
            gamma_values = [0.001, 0.01, 0.1, 1, 10]
        else:
            gamma_values = [0.001, 0.01, 0.1, 1, 10]

        # Create meshgrid of hyperparameters
        C_mesh, gamma_mesh = np.meshgrid(C_values, gamma_values)
        scores = np.zeros_like(C_mesh, dtype=float)

        # Evaluate each combination
        for i, gamma in enumerate(gamma_values):
            for j, C in enumerate(C_values):
                model = SVC(kernel='rbf', C=C, gamma=gamma, class_weight='balanced')
                model.fit(X_train, y_train)

                val_pred = model.predict(X_val)
                val_f1 = f1_score(y_val, val_pred, zero_division=0)

                scores[i, j] = val_f1

        # Plot heatmap
        plt.figure(figsize=(10, 8))
        sns.heatmap(scores, annot=True, fmt='.3f', cmap='viridis',
                    xticklabels=C_values, yticklabels=gamma_values)
        plt.xlabel('C (Regularization parameter)')
        plt.ylabel('gamma')
        plt.title('RBF SVM: Validation F1 Score for Different C and gamma Values')

        # Try to mark the best parameters
        if isinstance(best_params['gamma'], (int, float)):
            try:
                best_gamma_idx = gamma_values.index(best_params['gamma'])
                best_C_idx = C_values.index(best_params['C'])
                plt.plot(best_C_idx + 0.5, best_gamma_idx + 0.5, 'ro', markersize=12,
                        markeredgecolor='white', markeredgewidth=2)
            except:
                pass

        plt.savefig('rbf_svm_hyperparameter_comparison.png', dpi=300, bbox_inches='tight')
        plt.show()

def compare_kernels(X_train, y_train, X_val, y_val, X_test, y_test):
    # Find best hyperparameters for each kernel
    best_models = {}
    best_params = {}
    val_f1_scores = {}

    # Get best parameters for each kernel type
    for kernel in ['linear', 'poly', 'rbf']:
        print(f"\n{'-'*50}")
        print(f"Finding best hyperparameters for {kernel} kernel")
        print(f"{'-'*50}")

        best_model, params, val_f1 = run_grid_search_for_kernel(X_train, y_train, X_val, y_val, kernel)
        best_models[kernel] = best_model
        best_params[kernel] = params
        val_f1_scores[kernel] = val_f1

    # Evaluate performance for each kernel with its best hyperparameters
    results = {}

    for kernel in ['linear', 'poly', 'rbf']:
        print(f"\n{'-'*50}")
        print(f"Evaluating SVM with {kernel} kernel and optimal hyperparameters")
        print(f"Best parameters: {best_params[kernel]}")
        print(f"{'-'*50}")

        model = best_models[kernel]

        # Evaluate on each dataset
        _, train_acc, train_prec, train_rec, train_f1 = evaluate_model(model, X_train, y_train, f"{kernel.upper()} Training Set")
        _, val_acc, val_prec, val_rec, val_f1 = evaluate_model(model, X_val, y_val, f"{kernel.upper()} Validation Set")
        test_pred, test_acc, test_prec, test_rec, test_f1 = evaluate_model(model, X_test, y_test, f"{kernel.upper()} Test Set")

        # Store results
        results[kernel] = {
            'train': {'accuracy': train_acc, 'precision': train_prec, 'recall': train_rec, 'f1': train_f1},
            'val': {'accuracy': val_acc, 'precision': val_prec, 'recall': val_rec, 'f1': val_f1},
            'test': {'accuracy': test_acc, 'precision': test_prec, 'recall': test_rec, 'f1': test_f1}
        }

        # Plot confusion matrix for test set
        plot_confusion_matrix(y_test, test_pred, f"{kernel.upper()} SVM Test Set")

        # Try to plot ROC curve and PR curve (may fail if highly imbalanced)
        try:
            plot_roc_curve(model, X_test, y_test, f"{kernel.upper()} SVM")
            plot_precision_recall_curve(model, X_test, y_test, f"{kernel.upper()} SVM")
        except:
            print(f"Could not generate ROC or PR curves for {kernel} kernel - likely due to highly imbalanced data")

        # Try to plot decision boundary if 2D
        if X_test.shape[1] == 2:
            plot_decision_boundary(model, X_test, y_test, f"{kernel.upper()} SVM")

        # Plot hyperparameter comparison
        plot_hyperparameter_comparison(X_train, y_train, X_val, y_val, kernel, best_params[kernel])

    # Compare F1 scores across kernels
    plt.figure(figsize=(12, 6))
    x = np.arange(3)
    width = 0.25

    # Extract metrics for plotting
    train_f1 = [results[k]['train']['f1'] for k in ['linear', 'poly', 'rbf']]
    val_f1 = [results[k]['val']['f1'] for k in ['linear', 'poly', 'rbf']]
    test_f1 = [results[k]['test']['f1'] for k in ['linear', 'poly', 'rbf']]

    # Plot bars
    plt.bar(x - width, train_f1, width, label='Training')
    plt.bar(x, val_f1, width, label='Validation')
    plt.bar(x + width, test_f1, width, label='Test')

    plt.xlabel('Kernel')
    plt.ylabel('F1 Score')
    plt.title('F1 Score Comparison Across Different Kernels')
    plt.xticks(x, ['Linear', 'Polynomial', 'RBF'])
    plt.ylim(0, 1)
    plt.legend()
    plt.grid(True, axis='y')
    plt.tight_layout()
    plt.savefig('kernel_comparison_f1.png', dpi=300, bbox_inches='tight')
    plt.show()

    # Compare accuracy across kernels
    plt.figure(figsize=(12, 6))

    # Extract metrics for plotting
    train_acc = [results[k]['train']['accuracy'] for k in ['linear', 'poly', 'rbf']]
    val_acc = [results[k]['val']['accuracy'] for k in ['linear', 'poly', 'rbf']]
    test_acc = [results[k]['test']['accuracy'] for k in ['linear', 'poly', 'rbf']]

    # Plot bars
    plt.bar(x - width, train_acc, width, label='Training')
    plt.bar(x, val_acc, width, label='Validation')
    plt.bar(x + width, test_acc, width, label='Test')

    plt.xlabel('Kernel')
    plt.ylabel('Accuracy')
    plt.title('Accuracy Comparison Across Different Kernels')
    plt.xticks(x, ['Linear', 'Polynomial', 'RBF'])
    plt.ylim(0, 1)
    plt.legend()
    plt.grid(True, axis='y')
    plt.tight_layout()
    plt.savefig('kernel_comparison_accuracy.png', dpi=300, bbox_inches='tight')
    plt.show()

    # Determine best overall model based on validation F1 score
    best_kernel = max(val_f1_scores, key=val_f1_scores.get)
    print(f"\nBest overall kernel based on validation F1 score: {best_kernel.upper()}")
    print(f"Best hyperparameters: {best_params[best_kernel]}")

    return results, best_models[best_kernel], best_kernel, best_params

## Tried Running AASIST

Oops....I couldnt run it...

In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!ln -s /content/drive/MyDrive/aasist /content/aasist

In [ ]:
!git clone https://github.com/clovaai/aasist.git
%cd aasist


In [ ]:
!pip install -r requirements.txt


In [ ]:
!pip install --upgrade numpy==1.23.5 torch==2.0.1 tensorflow==2.12.0


In [ ]:
import gdown
gdown.download('https://drive.google.com/uc?id=MODEL_ID', 'aasist.pth')

# !wget https://github.com/clovaai/aasist/releases/download/v1.0/model_weights.pth


In [ ]:
import torch
from models import AASIST

model = AASIST()
model.load_state_dict(torch.load('model_weights.pth'))
model.to('cuda').eval()


## KNN (Bholendra Tripathi )

In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Load dataset
df = pd.read_csv('train.csv')

# Drop non-numeric and irrelevant columns
df = df.drop(columns=['id', 'filename', 'path'], errors='ignore')

# Separate features and target
y = df['label']
X = df.drop(columns=['label'])

# Keep only numeric columns
X = X.select_dtypes(include='number')

# Train on the entire dataset
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X, y)

# Predict on the same full dataset
y_pred = knn.predict(X)

# Evaluation
print("KNN Evaluation on Entire Dataset:")
print("Accuracy: ", round(accuracy_score(y, y_pred), 4))
print("Precision:", round(precision_score(y, y_pred, zero_division=0), 4))
print("Recall:   ", round(recall_score(y, y_pred), 4))
print("F1 Score: ", round(f1_score(y, y_pred), 4))

# Classification Report
print("\nClassification Report:\n", classification_report(y, y_pred))

# Confusion Matrix
cm = confusion_matrix(y, y_pred)
print("\nConfusion Matrix:\n", cm)

# Plot Confusion Matrix
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Fake', 'Real'], yticklabels=['Fake', 'Real'])
plt.title('Confusion Matrix (All Samples)')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.tight_layout()
plt.show()


## SVM (Bholendra Tripathi)

### SVM without Data modification

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load CSV
df = pd.read_csv("train.csv")

# Drop non-numeric columns (except for 'label')
numeric_df = df.select_dtypes(include=["number"])
X = numeric_df.drop(columns=["label"], errors='ignore')
y = df["label"]  # Use the original label column

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define kernels
kernels = ["linear", "poly", "rbf", "sigmoid"]

# Train and evaluate SVM for each kernel
for kernel in kernels:
    print(f"\nSVM with {kernel} kernel:")
    model = SVC(kernel=kernel)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, zero_division=1)
    recall = recall_score(y_test, y_pred, zero_division=1)
    f1 = f1_score(y_test, y_pred, zero_division=1)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")


### SVM with modified Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, roc_curve, auc, log_loss, hinge_loss
)
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from scipy.special import expit  # sigmoid

# Load CSV
df = pd.read_csv("train.csv")

# Drop non-numeric columns (except for 'label')
numeric_df = df.select_dtypes(include=["number"])
X = numeric_df.drop(columns=["label"], errors='ignore')
y = df["label"]

# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Dictionary to store results
results_dict = {}

# Define "kernels" via loss functions
losses = {
    "hinge (SVM)": "hinge",
    "logistic": "log_loss",
    "modified_huber": "modified_huber",
    "squared_hinge": "squared_hinge"
}

def train_plot_evaluate(X_train, X_test, y_train, y_test, technique):
    kernel_scores = []

    for label, loss_func in losses.items():
        print(f"\n[{technique}] Training with loss: {label}")
        clf = SGDClassifier(loss=loss_func, max_iter=1, tol=None, warm_start=True,
                            learning_rate='optimal', random_state=42)

        train_loss = []
        train_acc = []

        for epoch in range(50):  # Simulate epochs
            clf.fit(X_train, y_train)
            pred_train = clf.predict(X_train)
            acc = accuracy_score(y_train, pred_train)

            if loss_func in ['log_loss', 'modified_huber']:
                probs = expit(clf.decision_function(X_train))
                loss = log_loss(y_train, probs)
            else:
                scores = clf.decision_function(X_train)
                loss = hinge_loss(y_train, scores)

            train_loss.append(loss)
            train_acc.append(acc)

        # Plot Loss vs Epoch
        plt.figure(figsize=(10, 4))
        plt.subplot(1, 2, 1)
        plt.plot(train_loss, label='Loss')
        plt.title(f"Loss vs Epoch [{label}]")
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.grid()

        # Plot Accuracy vs Epoch
        plt.subplot(1, 2, 2)
        plt.plot(train_acc, label='Accuracy', color='green')
        plt.title(f"Accuracy vs Epoch [{label}]")
        plt.xlabel("Epoch")
        plt.ylabel("Accuracy")
        plt.grid()
        plt.tight_layout()
        plt.show()

        # Final Evaluation
        y_pred = clf.predict(X_test)
        y_score = clf.decision_function(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, zero_division=1)
        recall = recall_score(y_test, y_pred, zero_division=1)
        f1 = f1_score(y_test, y_pred, zero_division=1)

        kernel_scores.append((label, accuracy, precision, recall, f1))

        # Confusion Matrix
        cm = confusion_matrix(y_test, y_pred)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.title(f"Confusion Matrix [{label}]")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.show()

        # ROC Curve
        fpr, tpr, _ = roc_curve(y_test, y_score)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f"{label} AUC={roc_auc:.2f}")

    # ROC Summary Plot
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"ROC Curve [{technique}]")
    plt.legend()
    plt.grid()
    plt.show()

    results_dict[technique] = kernel_scores

# ==== 1. OVERSAMPLING ====
print("\n===== OVERSAMPLING =====")
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
X_train_os, y_train_os = SMOTE().fit_resample(X_train, y_train)
train_plot_evaluate(X_train_os, X_test, y_train_os, y_test, "Oversampling")

# ==== 2. UNDER-SAMPLING ====
print("\n===== UNDER-SAMPLING =====")
X_train_us, y_train_us = RandomUnderSampler().fit_resample(X_train, y_train)
train_plot_evaluate(X_train_us, X_test, y_train_us, y_test, "Under-sampling")

# ==== 3. CLASS WEIGHT ====
print("\n===== CLASS WEIGHT BIASING =====")
train_plot_evaluate(X_train, X_test, y_train, y_test, "ClassWeight")

# ==== 4. PCA ====
for n_comp in [50, 100, 200]:
    if n_comp >= X.shape[1]:
        continue
    print(f"\n===== PCA ({n_comp}) =====")
    pca = PCA(n_components=n_comp)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)
    train_plot_evaluate(X_train_pca, X_test_pca, y_train, y_test, f"PCA_{n_comp}")

# ==== BAR PLOTS ====
for technique, values in results_dict.items():
    df_bar = pd.DataFrame(values, columns=["Model", "Accuracy", "Precision", "Recall", "F1"])
    df_bar.set_index("Model", inplace=True)
    df_bar.plot(kind="bar", figsize=(10, 6), ylim=(0, 1), title=f"{technique} Performance")
    plt.grid(True, axis='y')
    plt.ylabel("Score")
    plt.tight_layout()
    plt.show()


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# Load CSV
df = pd.read_csv("train.csv")

# Drop non-numeric columns (except for 'label')
numeric_df = df.select_dtypes(include=["number"])
X = numeric_df.drop(columns=["label"], errors='ignore')
y = df["label"]

# Function to evaluate model
def evaluate(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, zero_division=1)
    recall = recall_score(y_test, y_pred, zero_division=1)
    f1 = f1_score(y_test, y_pred, zero_division=1)
    return accuracy, precision, recall, f1

# Define kernels
kernels = ["linear"]

# ---- 1. Oversampling using SMOTE ----
print("\n------ 1. Oversampling using SMOTE ------")
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

for kernel in kernels:
    print(f"\nSVM with {kernel} kernel:")
    model = SVC(kernel=kernel)
    model.fit(X_train_res, y_train_res)
    results = evaluate(model, X_test, y_test)
    print(f"Accuracy: {results[0]:.4f}, Precision: {results[1]:.4f}, Recall: {results[2]:.4f}, F1: {results[3]:.4f}")

# ---- 2. Under-sampling ----
print("\n------ 2. Under-sampling ------")
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
undersample = RandomUnderSampler(random_state=42)
X_train_res, y_train_res = undersample.fit_resample(X_train, y_train)

for kernel in kernels:
    print(f"\nSVM with {kernel} kernel:")
    model = SVC(kernel=kernel)
    model.fit(X_train_res, y_train_res)
    results = evaluate(model, X_test, y_test)
    print(f"Accuracy: {results[0]:.4f}, Precision: {results[1]:.4f}, Recall: {results[2]:.4f}, F1: {results[3]:.4f}")

# ---- 3. Class Weight Biasing ----
print("\n------ 3. Class Weight Biasing ------")
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

for kernel in kernels:
    print(f"\nSVM with {kernel} kernel and class_weight='balanced':")
    model = SVC(kernel=kernel, class_weight='balanced')
    model.fit(X_train, y_train)
    results = evaluate(model, X_test, y_test)
    print(f"Accuracy: {results[0]:.4f}, Precision: {results[1]:.4f}, Recall: {results[2]:.4f}, F1: {results[3]:.4f}")

# ---- 4. PCA ----
print("\n------ 4. PCA (n_components = 50, 100, 200) ------")
pca_components = [50, 100, 200]
X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

for n_comp in pca_components:
    print(f"\n-- PCA with {n_comp} components --")
    try:
        pca = PCA(n_components=n_comp)
        X_train_pca = pca.fit_transform(X_train_full)
        X_test_pca = pca.transform(X_test_full)
    except ValueError:
        print(f"Skipped: PCA({n_comp}) > feature count")
        continue

    for kernel in kernels:
        print(f"\nSVM with {kernel} kernel on PCA-reduced data:")
        model = SVC(kernel=kernel)
        model.fit(X_train_pca, y_train_full)
        results = evaluate(model, X_test_pca, y_test_full)
        print(f"Accuracy: {results[0]:.4f}, Precision: {results[1]:.4f}, Recall: {results[2]:.4f}, F1: {results[3]:.4f}")


## AdaBoost (Sailaja)

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[5]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score, f1_score, recall_score, confusion_matrix, roc_curve, auc,precision_recall_curve


# In[6]:


train_df = pd.read_csv("train.csv")
val_df = pd.read_csv("dev.csv")
test_df = pd.read_csv("eval.csv")


# In[7]:


X_train = train_df.iloc[:, 2:].values
y_train = train_df["label"].values

X_val = val_df.iloc[:, 2:].values
y_val = val_df["label"].values

X_test = test_df.iloc[:, 2:].values
y_test = test_df["label"].values


# In[8]:


from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

model = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=3),  # directly inside
    n_estimators=50,
    learning_rate=1.0,
    random_state=42
)


# Train the model
model.fit(X_train, y_train)


# In[9]:


y_train_pred = model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
train_recall = recall_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred)
y_train_probs = model.predict_proba(X_train)[:, 1]
def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    return eer
train_eer = compute_eer(y_train, y_train_probs)

print("\nTraining Results:")
print(f"Accuracy: {train_accuracy:.4f}")
print(f"F1 Score: {train_f1:.4f}")
print(f"Recall Score: {train_recall:.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_train, y_train_pred))
print(f"Equal Error Rate: {train_eer:.4f}")


# In[10]:


#####VALIDATION SET
y_val_pred = model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
val_recall=recall_score(y_val,y_val_pred)
val_f1 = f1_score(y_val, y_val_pred)
y_probs = model.predict_proba(X_val)[:, 1]
def compute_eer(y_true, y_scores):
  fpr, tpr, thresholds = roc_curve(y_true, y_scores)
  fnr = 1 - tpr
  eer_index = np.nanargmin(np.abs(fnr - fpr))
  eer = (fpr[eer_index] + fnr[eer_index]) / 2
  return eer
print("Validation Results:")
print(f"Accuracy: {val_accuracy:.4f}")
print(f"F1 Score: {val_f1:.4f}")
print(f"Recall Score: {val_recall:.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_val_pred))
eer = compute_eer(y_val, y_probs)
print(f"Equal Error Rate: {eer:.4f}")


# In[11]:


y_test_pred = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)
test_recall=recall_score(y_test,y_test_pred)
y_probs_test = model.predict_proba(X_test)[:, 1]
def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    return eer

print("\nTest (Evaluation) Results:")
print(f"Accuracy: {test_accuracy:.4f}")
print(f"F1 Score: {test_f1:.4f}")
print(f"Recall Score: {test_recall:.4f}")
eer_test = compute_eer(y_test, y_probs_test)
print(f"Equal Error Rate: {eer_test:.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))


# In[12]:


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import (
    confusion_matrix,
    roc_curve,
    precision_recall_curve
)


def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    threshold = thresholds[eer_index]
    return eer, fpr, tpr, threshold

def plot_graphs_only(model, X, y, set_name, row, fig):
    y_pred = model.predict(X)
    y_probs = model.predict_proba(X)[:, 1]

    cm = confusion_matrix(y, y_pred)
    eer, fpr, tpr, eer_threshold = compute_eer(y, y_probs)
    precision, recall, _ = precision_recall_curve(y, y_probs)

    # Plot 1: Confusion Matrix
    ax1 = fig.add_subplot(3, 3, 3*row + 1)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax1, cbar=False)
    ax1.set_title(f"{set_name} - Confusion Matrix")
    ax1.set_xlabel("Predicted")
    ax1.set_ylabel("Actual")

    # Plot 2: ROC Curve
    ax2 = fig.add_subplot(3, 3, 3*row + 2)
    ax2.plot(fpr, tpr, label="ROC Curve")
    ax2.plot([0, 1], [0, 1], 'k--', alpha=0.6)
    ax2.axvline(eer, color='red', linestyle='--', label=f"EER = {eer:.4f}")
    ax2.set_title(f"{set_name} - ROC Curve")
    ax2.set_xlabel("False Positive Rate")
    ax2.set_ylabel("True Positive Rate")
    ax2.legend()
    ax2.grid(True)

    # Plot 3: Precision-Recall Curve
    ax3 = fig.add_subplot(3, 3, 3*row + 3)
    ax3.plot(recall, precision, label="PR Curve")
    ax3.set_title(f"{set_name} - Precision-Recall Curve")
    ax3.set_xlabel("Recall")
    ax3.set_ylabel("Precision")
    ax3.legend()
    ax3.grid(True)



fig = plt.figure(figsize=(18, 12))
plot_graphs_only(model, X_train, y_train, "Training (Oversampled)", 0, fig)
plot_graphs_only(model, X_val, y_val, "Validation", 1, fig)
plot_graphs_only(model, X_test, y_test, "Test", 2, fig)
plt.tight_layout()
plt.show()


# In[15]:


########################SMOTING

from imblearn.over_sampling import SMOTE
import pandas as pd


smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)


OVERtrain_balanced_df = pd.DataFrame(X_train_resampled)
OVERtrain_balanced_df['label'] = y_train_resampled

# Check new label distribution
print(OVERtrain_balanced_df['label'].value_counts())


# In[ ]:


from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

# Define AdaBoost with DecisionTree base and max_depth=3
over_xg_boost = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=3),
    n_estimators=50,
    learning_rate=0.1,
    random_state=42
)


over_xg_boost.fit(X_train_resampled, y_train_resampled)

# Make predictions
over_y_train_pred = over_xg_boost.predict(X_train_resampled)
over_y_val_pred = over_xg_boost.predict(X_val)
over_y_test_pred = over_xg_boost.predict(X_test)


# In[ ]:


def evaluate(X_eval, y_true, y_pred, dataset_name):
    from sklearn.metrics import accuracy_score, f1_score, recall_score, confusion_matrix, roc_curve
    import numpy as np

    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)

    y_score = over_xg_boost.predict_proba(X_eval)[:, 1]
    fpr, tpr, _ = roc_curve(y_true, y_score)
    eer = fpr[np.nanargmin(np.absolute((1 - tpr) - fpr))]

    print(f"{dataset_name} Results:")
    print("Accuracy:", round(acc, 4))
    print("F1 Score:", round(f1, 4))
    print("Recall:", round(rec, 4))
    print("Confusion Matrix:\n", cm)
    print("Equal Error Rate (EER):", round(eer, 4))
    print()


evaluate(X_train_resampled, y_train_resampled, over_y_train_pred, "Training")
evaluate(X_val, y_val, over_y_val_pred, "Validation")
evaluate(X_test, y_test, over_y_test_pred, "Test")


# In[ ]:


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import (
    confusion_matrix,
    roc_curve,
    precision_recall_curve
)


def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    threshold = thresholds[eer_index]
    return eer, fpr, tpr, threshold

def plot_graphs_only(model, X, y, set_name, row, fig):
    y_pred = model.predict(X)
    y_probs = model.predict_proba(X)[:, 1]

    cm = confusion_matrix(y, y_pred)
    eer, fpr, tpr, eer_threshold = compute_eer(y, y_probs)
    precision, recall, _ = precision_recall_curve(y, y_probs)

    # Plot 1: Confusion Matrix
    ax1 = fig.add_subplot(3, 3, 3*row + 1)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax1, cbar=False)
    ax1.set_title(f"{set_name} - Confusion Matrix")
    ax1.set_xlabel("Predicted")
    ax1.set_ylabel("Actual")

    # Plot 2: ROC Curve
    ax2 = fig.add_subplot(3, 3, 3*row + 2)
    ax2.plot(fpr, tpr, label="ROC Curve")
    ax2.plot([0, 1], [0, 1], 'k--', alpha=0.6)
    ax2.axvline(eer, color='red', linestyle='--', label=f"EER = {eer:.4f}")
    ax2.set_title(f"{set_name} - ROC Curve")
    ax2.set_xlabel("False Positive Rate")
    ax2.set_ylabel("True Positive Rate")
    ax2.legend()
    ax2.grid(True)

    # Plot 3: Precision-Recall Curve
    ax3 = fig.add_subplot(3, 3, 3*row + 3)
    ax3.plot(recall, precision, label="PR Curve")
    ax3.set_title(f"{set_name} - Precision-Recall Curve")
    ax3.set_xlabel("Recall")
    ax3.set_ylabel("Precision")
    ax3.legend()
    ax3.grid(True)



fig = plt.figure(figsize=(18, 12))
plot_graphs_only(over_xg_boost, X_train, y_train, "Training (Oversampled)", 0, fig)
plot_graphs_only(over_xg_boost, X_val, y_val, "Validation", 1, fig)
plot_graphs_only(over_xg_boost, X_test, y_test, "Test", 2, fig)
plt.tight_layout()
plt.show()


# In[20]:


##################UNDERSAMPLING
print(train_df['label'].value_counts())
class_0 = train_df[train_df['label'] == 0]
class_1 = train_df[train_df['label'] == 1]
class_0_undersampled = class_0.sample(n=len(class_1), random_state=42)
train_balanced_df = pd.concat([class_0_undersampled, class_1], axis=0)
train_balanced_df = train_balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)
print("Balanced Training Set:")
print(train_balanced_df['label'].value_counts())


# In[21]:


X_train_undersampling = train_balanced_df.iloc[:, 2:].values
y_train_undersampling = train_balanced_df["label"].values
under_xg_boost =AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=3),
    n_estimators=50,
    learning_rate=0.1,
    random_state=42
)
under_xg_boost.fit(X_train_undersampling, y_train_undersampling)


# In[22]:


under_y_train_pred = under_xg_boost.predict(X_train_undersampling)
under_y_val_pred = under_xg_boost.predict(X_val)
under_y_test_pred = under_xg_boost.predict(X_test)


# In[13]:


def evaluate_under(y_true, y_pred, X_data, dataset_name):
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    fpr, tpr, _ = roc_curve(y_true, under_xg_boost.predict_proba(X_data)[:, 1])
    eer = fpr[np.nanargmin(np.absolute((1 - tpr) - fpr))]

    print(f"{dataset_name} Results:")
    print(f"Accuracy: {acc:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Recall: {rec:.4f}")
    print(f"Confusion Matrix:\n {cm}")
    print(f"Equal Error Rate (EER): {eer:.4f}\n")

evaluate_under(y_train_undersampling, under_y_train_pred, X_train_undersampling, "Training")
evaluate_under(y_val, under_y_val_pred, X_val, "Validation")
evaluate_under(y_test, under_y_test_pred, X_test, "Test (Evaluation)")


# In[ ]:


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import (
    confusion_matrix,
    roc_curve,
    precision_recall_curve
)


def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    threshold = thresholds[eer_index]
    return eer, fpr, tpr, threshold

def plot_graphs_only(model, X, y, set_name, row, fig):
    y_pred = model.predict(X)
    y_probs = model.predict_proba(X)[:, 1]

    cm = confusion_matrix(y, y_pred)
    eer, fpr, tpr, eer_threshold = compute_eer(y, y_probs)
    precision, recall, _ = precision_recall_curve(y, y_probs)

    # Plot 1: Confusion Matrix
    ax1 = fig.add_subplot(3, 3, 3*row + 1)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax1, cbar=False)
    ax1.set_title(f"{set_name} - Confusion Matrix")
    ax1.set_xlabel("Predicted")
    ax1.set_ylabel("Actual")

    # Plot 2: ROC Curve
    ax2 = fig.add_subplot(3, 3, 3*row + 2)
    ax2.plot(fpr, tpr, label="ROC Curve")
    ax2.plot([0, 1], [0, 1], 'k--', alpha=0.6)
    ax2.axvline(eer, color='red', linestyle='--', label=f"EER = {eer:.4f}")
    ax2.set_title(f"{set_name} - ROC Curve")
    ax2.set_xlabel("False Positive Rate")
    ax2.set_ylabel("True Positive Rate")
    ax2.legend()
    ax2.grid(True)

    # Plot 3: Precision-Recall Curve
    ax3 = fig.add_subplot(3, 3, 3*row + 3)
    ax3.plot(recall, precision, label="PR Curve")
    ax3.set_title(f"{set_name} - Precision-Recall Curve")
    ax3.set_xlabel("Recall")
    ax3.set_ylabel("Precision")
    ax3.legend()
    ax3.grid(True)



fig = plt.figure(figsize=(18, 12))
plot_graphs_only(under_xg_boost, X_train_undersampling, y_train_undersampling, "Training (Oversampled)", 0, fig)
plot_graphs_only(under_xg_boost, X_val, y_val, "Validation", 1, fig)
plot_graphs_only(under_xg_boost, X_test, y_test, "Test", 2, fig)
plt.tight_layout()
plt.show()


# In[23]:


from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

pca = PCA(n_components=50)
X_train_pca = pca.fit_transform(X_train_scaled)
X_val_pca = pca.transform(X_val_scaled)
X_test_pca = pca.transform(X_test_scaled)


model_pca = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=2),
    n_estimators=100,
    learning_rate=1.0,
    random_state=42
)

model_pca.fit(X_train_pca, y_train)


train_preds= model_pca.predict(X_train_pca)
val_preds = model_pca.predict(X_val_pca)
test_preds = model_pca.predict(X_test_pca)


# In[26]:


from sklearn.metrics import accuracy_score, classification_report, f1_score, recall_score, confusion_matrix, roc_curve
import numpy as np

def evaluate_pca(y_true, y_pred, X_data, dataset_name):
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)


    fpr, tpr, _ = roc_curve(y_true, model_pca.predict_proba(X_data)[:, 1])
    eer = fpr[np.nanargmin(np.absolute((1 - tpr) - fpr))]

    print(f"{dataset_name} Results:")
    print(f"Accuracy: {acc:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Recall: {rec:.4f}")
    print(f"Confusion Matrix:\n {cm}")
    print(f"Equal Error Rate (EER): {eer:.4f}\n")
evaluate_pca(y_train, train_preds, X_train_pca, "Training")
evaluate_pca(y_val, val_preds, X_val_pca, "Validation")
evaluate_pca(y_test, test_preds, X_test_pca, "Test (Evaluation)")


# In[27]:


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def plot_all_graphs(model, X_sets, y_sets, set_names):
    num_sets = len(X_sets)
    fig, axes = plt.subplots(nrows=num_sets, ncols=3, figsize=(18, 4 * num_sets))

    for i, (X, y, name) in enumerate(zip(X_sets, y_sets, set_names)):
        y_pred = model.predict(X)
        y_probs = model.predict_proba(X)[:, 1]

        # 1. Actual Labels
        sns.countplot(x=y, hue=y, palette='cool', ax=axes[i, 0], legend=False)
        axes[i, 0].set_title(f'{name} - Actual Labels')
        axes[i, 0].set_xlabel('Class')
        axes[i, 0].set_ylabel('Count')

        # 2. Predicted Labels
        sns.countplot(x=y_pred, hue=y_pred, palette='magma', ax=axes[i, 1], legend=False)
        axes[i, 1].set_title(f'{name} - Predicted Labels')
        axes[i, 1].set_xlabel('Class')
        axes[i, 1].set_ylabel('Count')

        # 3. Scatter Plot
        sns.scatterplot(x=np.arange(len(y_probs)), y=y_probs, hue=y, palette='Set2', alpha=0.7, ax=axes[i, 2])
        axes[i, 2].set_title(f'{name} - Probabilities vs True Labels')
        axes[i, 2].set_xlabel('Sample Index')
        axes[i, 2].set_ylabel('Predicted Probability')
        axes[i, 2].legend(title='True Label')

    plt.tight_layout()
    plt.show()


# Example call
plot_all_graphs(
    model,
    [X_train, X_val, X_test],
    [y_train, y_val, y_test],
    ["Training", "Validation", "Test"]
)


# In[ ]:





# In[28]:


#############SOMTING
plot_all_graphs(
    over_xg_boost,
    [X_train_resampled, X_val, X_test],
    [y_train_resampled, y_val, y_test],
    ["Training", "Validation", "Test"]
)


# In[29]:


########UNDESAMPLING
plot_all_graphs(
    under_xg_boost,
    [X_train_undersampling, X_val, X_test],
    [y_train_undersampling, y_val, y_test],
    ["Training", "Validation", "Test"]
)


# In[30]:


##########PCA
plot_all_graphs(
    model_pca,
    [X_train_pca, X_val_pca, X_test_pca],
    [y_train, y_val, y_test],
    ["Training", "Validation", "Test"]
)


# In[15]:


import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

def plot_confusion_matrices(y_tr, y_tr_pred, y_val, y_val_pred, y_test, y_test_pred):
    titles = ["Training Set", "Validation Set", "Test Set"]
    datasets = [(y_tr, y_tr_pred), (y_val, y_val_pred), (y_test, y_test_pred)]

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    for ax, (y_true, y_pred), title in zip(axes, datasets, titles):
        cm = confusion_matrix(y_true, y_pred)
        sns.heatmap(cm, annot=True, fmt="d", cmap="coolwarm", cbar=False, ax=ax)
        ax.set_title(f"Confusion Matrix - {title}")
        ax.set_xlabel("Predicted")
        ax.set_ylabel("Actual")

    plt.tight_layout()
    plt.show()


# In[32]:


########MODEL
plot_confusion_matrices(y_train, y_train_pred, y_val, y_val_pred, y_test, y_test_pred)


# In[33]:


plot_confusion_matrices(y_train_resampled, over_y_train_pred, y_val, over_y_val_pred, y_test, over_y_test_pred)


# In[16]:


plot_confusion_matrices(y_train_undersampling, under_y_train_pred, y_val, under_y_val_pred, y_test, under_y_test_pred)


# In[35]:


plot_confusion_matrices(y_train, train_preds, y_val, val_preds, y_test, test_preds)


# In[36]:


import matplotlib.pyplot as plt
import numpy as np

# Techniques and their test accuracies
techniques = ["Original", "Oversampling", "Undersampling", "PCA"]
accuracies = [0.82, 0.851, 0.78, 0.80]

x = np.arange(len(techniques))

# Plotting
plt.figure(figsize=(8, 6))
plt.bar(x, accuracies, width=0.5, color=["skyblue", "lightgreen", "salmon", "orange"])

# Labeling
plt.ylabel("Test Accuracy")
plt.title("Test Accuracy vs Data Technique")
plt.xticks(x, techniques)
plt.ylim(0.7, 0.9)
plt.grid(axis="y", linestyle="--", alpha=0.7)

plt.tight_layout()
plt.show()


# In[17]:


import matplotlib.pyplot as plt
import numpy as np

# Define the confusion matrix for each case
confusion_matrices = {
    "Original Training": [[22579, 221], [2260, 320]],
    "Original Validation": [[21986, 310], [2389, 159]],
    "Original Test": [[63882, 0], [7355, 0]],

    "Oversampling Training": [[13708, 9092], [2563, 20237]],
    "Oversampling Validation": [[13488, 8808], [761, 1787]],
    "Oversampling Test": [[63716, 166], [7353, 2]],

    "Undersampling Training": [[1724, 856], [585, 1995]],
    "Undersampling Validation": [[14299, 7997], [789, 1759]],
    "Undersampling Test": [[63346, 536], [7257, 98]],

    "PCA Reduction Training": [[22715, 85], [2543, 37]],
    "PCA Reduction validation": [[22231, 65], [2519, 29]],
    "PCA Reduction Test": [[63882, 0], [7355, 0]]
}


# Function to calculate accuracy for label 0 and label 1
def calculate_accuracies(conf_matrix):
    # Label 0 accuracy = True Negatives / (True Negatives + False Positives)
    accuracy_label_0 = conf_matrix[0][0] / (conf_matrix[0][0] + conf_matrix[0][1])
    # Label 1 accuracy = True Positives / (True Positives + False Negatives)
    accuracy_label_1 = conf_matrix[1][1] / (conf_matrix[1][0] + conf_matrix[1][1])
    return accuracy_label_0, accuracy_label_1

# Prepare data for plotting
labels = list(confusion_matrices.keys())
accuracies_label_0 = []
accuracies_label_1 = []

for label in labels:
    conf_matrix = confusion_matrices[label]
    acc_label_0, acc_label_1 = calculate_accuracies(conf_matrix)
    accuracies_label_0.append(acc_label_0)
    accuracies_label_1.append(acc_label_1)

# Plotting the accuracies
x = np.arange(len(labels))  # Label positions

fig, ax = plt.subplots(figsize=(12, 8))
width = 0.35  # Bar width

# Set colors for the bars (one color for label 0, another for label 1)
color_label_0 = 'skyblue'
color_label_1 = 'orange'

# Plotting bars for Label 0 and Label 1
bars_label_0 = ax.bar(x - width/2, accuracies_label_0, width, label='Accuracy for Label 0', color=color_label_0)
bars_label_1 = ax.bar(x + width/2, accuracies_label_1, width, label='Accuracy for Label 1', color=color_label_1)

# Labeling and formatting
ax.set_xlabel('Model Configuration')
ax.set_ylabel('Accuracy')
ax.set_title('Accuracy for Label 0 and Label 1 in Different Models')
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=45, ha='right')
ax.legend()

# Display the plot
plt.tight_layout()
plt.show()


# In[ ]:


from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score


adaboost = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(random_state=42),
    random_state=42
)

# Updated param_grid for newer versions (use 'estimator__' prefix)
param_grid = {
    'estimator__max_depth': [3, 5, 6],
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.5],
}

# Grid search for best parameters
grid_search = GridSearchCV(adaboost, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Best parameters found by GridSearchCV
print("Best Parameters:", grid_search.best_params_)

# Evaluate on test set using best model
best_adaboost = grid_search.best_estimator_
y_pred = best_adaboost.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy with Best Model: {accuracy:.4f}")


# In[23]:


pip install --upgrade scikit-learn


# In[ ]:





# In[ ]:






## Gradient Boost (Sailaja)

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, confusion_matrix, roc_curve, auc,precision_recall_curve


# In[2]:


train_df = pd.read_csv("train.csv")
val_df = pd.read_csv("dev.csv")
test_df = pd.read_csv("eval.csv")


# In[3]:


X_train = train_df.iloc[:, 2:].values
y_train = train_df["label"].values

X_val = val_df.iloc[:, 2:].values
y_val = val_df["label"].values

X_test = test_df.iloc[:, 2:].values
y_test = test_df["label"].values


# In[4]:


gb_original = GradientBoostingClassifier(n_estimators=40, learning_rate=0.1, max_depth=4, random_state=42)
gb_original.fit(X_train, y_train)


# In[5]:



y_train_pred = gb_original.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
train_recall = recall_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred)
y_train_probs = gb_original.predict_proba(X_train)[:, 1]
def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    return eer
train_eer = compute_eer(y_train, y_train_probs)

print("\nTraining Results:")
print(f"Accuracy: {train_accuracy:.4f}")
print(f"F1 Score: {train_f1:.4f}")
print(f"Recall Score: {train_recall:.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_train, y_train_pred))
print(f"Equal Error Rate: {train_eer:.4f}")


# In[6]:


#####VALIDATION SET
y_val_pred = gb_original.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
val_recall=recall_score(y_val,y_val_pred)
val_f1 = f1_score(y_val, y_val_pred)
y_probs = gb_original.predict_proba(X_val)[:, 1]
def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    return eer
print("Validation Results:")
print(f"Accuracy: {val_accuracy:.4f}")
print(f"F1 Score: {val_f1:.4f}")
print(f"Recall Score: {val_recall:.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_val_pred))
eer = compute_eer(y_val, y_probs)
print(f"Equal Error Rate: {eer:.4f}")


# In[7]:


y_test_pred = gb_original.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)
test_recall=recall_score(y_test,y_test_pred)
y_probs_test = gb_original.predict_proba(X_test)[:, 1]
def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    return eer

print("\nTest (Evaluation) Results:")
print(f"Accuracy: {test_accuracy:.4f}")
print(f"F1 Score: {test_f1:.4f}")
print(f"Recall Score: {test_recall:.4f}")
eer_test = compute_eer(y_test, y_probs_test)
print(f"Equal Error Rate: {eer_test:.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))


# In[ ]:





# In[33]:


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import (
    confusion_matrix,
    roc_curve,
    precision_recall_curve
)

def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    threshold = thresholds[eer_index]
    return eer, fpr, tpr, threshold

def plot_graphs_only(model, X, y, set_name, row, fig):
    y_pred = model.predict(X)
    y_probs = model.predict_proba(X)[:, 1]

    cm = confusion_matrix(y, y_pred)
    eer, fpr, tpr, eer_threshold = compute_eer(y, y_probs)
    precision, recall, _ = precision_recall_curve(y, y_probs)

    # Plot 1: Confusion Matrix
    ax1 = fig.add_subplot(3, 3, 3*row + 1)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax1, cbar=False)
    ax1.set_title(f"{set_name} - Confusion Matrix")
    ax1.set_xlabel("Predicted")
    ax1.set_ylabel("Actual")

    # Plot 2: ROC Curve
    ax2 = fig.add_subplot(3, 3, 3*row + 2)
    ax2.plot(fpr, tpr, label="ROC Curve")
    ax2.plot([0, 1], [0, 1], 'k--', alpha=0.6)
    ax2.axvline(eer, color='red', linestyle='--', label=f"EER = {eer:.4f}")
    ax2.set_title(f"{set_name} - ROC Curve")
    ax2.set_xlabel("False Positive Rate")
    ax2.set_ylabel("True Positive Rate")
    ax2.legend()
    ax2.grid(True)

    # Plot 3: Precision-Recall Curve
    ax3 = fig.add_subplot(3, 3, 3*row + 3)
    ax3.plot(recall, precision, label="PR Curve")
    ax3.set_title(f"{set_name} - Precision-Recall Curve")
    ax3.set_xlabel("Recall")
    ax3.set_ylabel("Precision")
    ax3.legend()
    ax3.grid(True)



fig = plt.figure(figsize=(18, 12))


plot_graphs_only(gb_original, X_train, y_train, "Training", 0, fig)
plot_graphs_only(gb_original, X_val, y_val, "Validation", 1, fig)
plot_graphs_only(gb_original, X_test, y_test, "Test", 2, fig)


plt.tight_layout()
plt.show()


# In[42]:


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def plot_all_graphs(model, X_sets, y_sets, set_names):
    num_sets = len(X_sets)
    fig, axes = plt.subplots(nrows=num_sets, ncols=3, figsize=(18, 4 * num_sets))

    for i, (X, y, name) in enumerate(zip(X_sets, y_sets, set_names)):
        y_pred = model.predict(X)
        y_probs = model.predict_proba(X)[:, 1]

        # 1. Actual Labels
        sns.countplot(x=y, hue=y, palette='cool', ax=axes[i, 0], legend=False)
        axes[i, 0].set_title(f'{name} - Actual Labels')
        axes[i, 0].set_xlabel('Class')
        axes[i, 0].set_ylabel('Count')

        # 2. Predicted Labels
        sns.countplot(x=y_pred, hue=y_pred, palette='magma', ax=axes[i, 1], legend=False)
        axes[i, 1].set_title(f'{name} - Predicted Labels')
        axes[i, 1].set_xlabel('Class')
        axes[i, 1].set_ylabel('Count')

        # 3. Scatter Plot
        sns.scatterplot(x=np.arange(len(y_probs)), y=y_probs, hue=y, palette='Set2', alpha=0.7, ax=axes[i, 2])
        axes[i, 2].set_title(f'{name} - Probabilities vs True Labels')
        axes[i, 2].set_xlabel('Sample Index')
        axes[i, 2].set_ylabel('Predicted Probability')
        axes[i, 2].legend(title='True Label')

    plt.tight_layout()
    plt.show()


# Example call
plot_all_graphs(
    gb_original,
    [X_train, X_val, X_test],
    [y_train, y_val, y_test],
    ["Training", "Validation", "Test"]
)


# In[18]:


#####UNDERSAMPLING
print(train_df['label'].value_counts())
class_0 = train_df[train_df['label'] == 0]
class_1 = train_df[train_df['label'] == 1]
class_0_undersampled = class_0.sample(n=len(class_1), random_state=42)
train_balanced_df = pd.concat([class_0_undersampled, class_1], axis=0)
train_balanced_df = train_balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)
print("Balanced Training Set:")
print(train_balanced_df['label'].value_counts())


# In[20]:


X_train_undersampling = train_balanced_df.iloc[:, 2:].values
y_train_undersampling = train_balanced_df["label"].values
gb_original_undersampling = GradientBoostingClassifier(n_estimators=40, learning_rate=0.1, max_depth=4, random_state=42)
gb_original_undersampling.fit(X_train_undersampling, y_train_undersampling)


# In[18]:


from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix, roc_curve
import numpy as np

under_y_train_pred = gb_original_undersampling.predict(X_train_undersampling)
under_y_train_probs = gb_original_undersampling.predict_proba(X_train_undersampling)[:, 1]

under_train_accuracy = accuracy_score(y_train_undersampling, under_y_train_pred)
under_train_recall = recall_score(y_train_undersampling, under_y_train_pred)
under_train_f1 = f1_score(y_train_undersampling, under_y_train_pred)


def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    return eer

under_train_eer = compute_eer(y_train_undersampling, under_y_train_probs)

under_conf_matrix = confusion_matrix(y_train_undersampling, under_y_train_pred)


print("Under - Training Results on Undersampled Data:")
print(f"Under - Accuracy: {under_train_accuracy:.4f}")
print(f"Under - F1 Score: {under_train_f1:.4f}")
print(f"Under - Recall Score: {under_train_recall:.4f}")
print("Under - Confusion Matrix:")
print(under_conf_matrix)
print(f"Under - Equal Error Rate: {under_train_eer:.4f}")


# In[19]:


#######VALIDATION SET AFTER UNDERSAMPLING
UNDERy_val_pred = gb_original_undersampling.predict(X_val)
UNDERval_accuracy = accuracy_score(y_val, UNDERy_val_pred)
UNDERval_recall=recall_score(y_val,UNDERy_val_pred)
UNDERval_f1 = f1_score(y_val, UNDERy_val_pred)
UNDERy_probs = gb_original_undersampling.predict_proba(X_val)[:, 1]
def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    return eer
print("Validation Results:")
print(f"Accuracy: {UNDERval_accuracy :.4f}")
print(f"F1 Score: {UNDERval_f1:.4f}")
print(f"Recall Score: {UNDERval_recall:.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_val, UNDERy_val_pred))
UNDEReer = compute_eer(y_val, y_probs)
print(f"Equal Error Rate: {UNDEReer:.4f}")


# In[20]:


###TEST SET AFTER UNDERSAMPLING
UNDERy_test_pred = gb_original_undersampling.predict(X_test)
UNDERtest_accuracy = accuracy_score(y_test, UNDERy_test_pred)
UNDERtest_f1 = f1_score(y_test, UNDERy_test_pred)
UNDERtest_recall=recall_score(y_test,UNDERy_test_pred)
UNDERy_probs_test = gb_original_undersampling.predict_proba(X_test)[:, 1]
def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    return eer

print("\nTest (Evaluation) Results:")
print(f"Accuracy: {UNDERtest_accuracy:.4f}")
print(f"F1 Score: {UNDERtest_f1:.4f}")
print(f"Recall Score: {UNDERtest_recall:.4f}")
UNDEReer_test = compute_eer(y_test, UNDERy_probs_test)
print(f"Equal Error Rate: {UNDEReer_test:.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, UNDERy_test_pred))


# In[12]:


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Define confusion matrices
conf_matrices = {
    "Training": np.array([[2121, 459],
                          [300,  2280]]),

    "Validation": np.array([[15096, 7200],
                            [768,   1780]]),

    "Test": np.array([[63683, 199],
                      [7316,   39]])
}

# Set up subplots
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for ax, (title, matrix) in zip(axes, conf_matrices.items()):
    sns.heatmap(matrix, annot=True, fmt='d', cmap='coolwarm', ax=ax)
    ax.set_title(f"{title} Set")
    ax.set_xlabel("Predicted Label")
    ax.set_ylabel("True Label")

plt.tight_layout()
plt.show()


# In[35]:


# Create the figure
fig = plt.figure(figsize=(18, 12))

# Add all sets
plot_graphs_only(gb_original_undersampling, X_train_undersampling, y_train_undersampling, "Training", 0, fig)
plot_graphs_only(gb_original_undersampling, X_val, y_val, "Validation", 1, fig)
plot_graphs_only(gb_original_undersampling, X_test, y_test, "Test", 2, fig)
# Layout and show
plt.tight_layout()
plt.show()


# In[43]:


plot_all_graphs(
    gb_original_undersampling,
    [X_train_undersampling, X_val, X_test],
    [y_train_undersampling, y_val, y_test],
    ["Training", "Validation", "Test"]
)


# In[44]:


####OVERSAMPLING

print("Original Class Distribution:")
print(train_df['label'].value_counts())

class_0 = train_df[train_df['label'] == 0]
class_1 = train_df[train_df['label'] == 1]

class_1_oversampled = class_1.sample(n=len(class_0), replace=True, random_state=42)

OVERtrain_balanced_df = pd.concat([class_0, class_1_oversampled], axis=0)

# Shuffle the combined dataset
OVERtrain_balanced_df = OVERtrain_balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Show the new class distribution
print("Balanced Training Set After Oversampling:")
print(OVERtrain_balanced_df['label'].value_counts())


# In[45]:


X_train_oversampling = OVERtrain_balanced_df.iloc[:, 2:].values
y_train_oversampling = OVERtrain_balanced_df["label"].values
gb_oversampling = GradientBoostingClassifier(n_estimators=30, learning_rate=0.1, max_depth=3, random_state=42)
gb_oversampling.fit(X_train_oversampling, y_train_oversampling)


# In[46]:


######VALIDATION METRICS AFTER OVERSAMPLING
OVERy_val_pred = gb_oversampling.predict(X_val)
OVERval_accuracy = accuracy_score(y_val, OVERy_val_pred)
OVERval_recall=recall_score(y_val,OVERy_val_pred)
OVERval_f1 = f1_score(y_val, OVERy_val_pred)
OVERy_probs = gb_oversampling.predict_proba(X_val)[:, 1]
def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    return eer
print("Validation Results:")
print(f"Accuracy: {OVERval_accuracy :.4f}")
print(f"F1 Score: {OVERval_f1:.4f}")
print(f"Recall Score: {OVERval_recall:.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_val, OVERy_val_pred))
OVEReer = compute_eer(y_val, OVERy_probs)
print(f"Equal Error Rate: {OVEReer:.4f}")


# In[47]:


###TEST SET AFTER OVERSAMPLING
OVERy_test_pred = gb_oversampling.predict(X_test)
OVERtest_accuracy = accuracy_score(y_test, OVERy_test_pred)
OVERtest_f1 = f1_score(y_test,OVERy_test_pred)
OVERtest_recall=recall_score(y_test,OVERy_test_pred)
OVERy_probs_test = gb_oversampling.predict_proba(X_test)[:, 1]
def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    return eer

print("\nTest (Evaluation) Results:")
print(f"Accuracy: {OVERtest_accuracy:.4f}")
print(f"F1 Score: {OVERtest_f1:.4f}")
print(f"Recall Score: {OVERtest_recall:.4f}")
OVEReer_test = compute_eer(y_test, OVERy_probs_test)
print(f"Equal Error Rate: {OVEReer_test:.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, OVERy_test_pred))


# In[48]:


plot_all_graphs(
    gb_oversampling,
    [X_train_oversampling, X_val, X_test],
    [y_train_oversampling, y_val, y_test],
    ["Training", "Validation", "Test"]
)


# In[52]:


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import (
    confusion_matrix,
    roc_curve,
    precision_recall_curve
)


def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    threshold = thresholds[eer_index]
    return eer, fpr, tpr, threshold

def plot_graphs_only(model, X, y, set_name, row, fig):
    y_pred = model.predict(X)
    y_probs = model.predict_proba(X)[:, 1]

    cm = confusion_matrix(y, y_pred)
    eer, fpr, tpr, eer_threshold = compute_eer(y, y_probs)
    precision, recall, _ = precision_recall_curve(y, y_probs)

    # Plot 1: Confusion Matrix
    ax1 = fig.add_subplot(3, 3, 3*row + 1)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax1, cbar=False)
    ax1.set_title(f"{set_name} - Confusion Matrix")
    ax1.set_xlabel("Predicted")
    ax1.set_ylabel("Actual")

    # Plot 2: ROC Curve
    ax2 = fig.add_subplot(3, 3, 3*row + 2)
    ax2.plot(fpr, tpr, label="ROC Curve")
    ax2.plot([0, 1], [0, 1], 'k--', alpha=0.6)
    ax2.axvline(eer, color='red', linestyle='--', label=f"EER = {eer:.4f}")
    ax2.set_title(f"{set_name} - ROC Curve")
    ax2.set_xlabel("False Positive Rate")
    ax2.set_ylabel("True Positive Rate")
    ax2.legend()
    ax2.grid(True)

    # Plot 3: Precision-Recall Curve
    ax3 = fig.add_subplot(3, 3, 3*row + 3)
    ax3.plot(recall, precision, label="PR Curve")
    ax3.set_title(f"{set_name} - Precision-Recall Curve")
    ax3.set_xlabel("Recall")
    ax3.set_ylabel("Precision")
    ax3.legend()
    ax3.grid(True)



fig = plt.figure(figsize=(18, 12))
plot_graphs_only(gb_oversampling, X_train_oversampling, y_train_oversampling, "Training (Oversampled)", 0, fig)
plot_graphs_only(gb_oversampling, X_val, y_val, "Validation", 1, fig)
plot_graphs_only(gb_oversampling, X_test, y_test, "Test", 2, fig)
plt.tight_layout()
plt.show()


# In[ ]:





# In[8]:


import matplotlib.pyplot as plt
import numpy as np


confusion_matrices = {
    "Original Training": [[22800, 0], [2557, 23]],
    "Original Validation": [[22295, 1], [2545, 3]],
    "Original Test": [[63033, 849], [7125, 230]],

    "Undersampling Training": [[2121, 459], [300, 2280]],
    "Undersampling Validation": [[15096, 7200], [768, 1780]],
    "Undersampling Test": [[63683, 199], [7316, 39]],

    "Oversampling Training": [[16408, 6392], [3422, 19378]],
    "Oversampling Validation": [[16054, 6242], [1031, 1517]],
    "Oversampling Test": [[63877, 5], [7354, 1]],

    "Class Weighting Training": [[15481, 7319], [598, 1982]],
    "Class Weighting Validation": [[15263, 7033], [905, 1643]],
    "Class Weighting Test": [[63815, 67], [7342, 13]]
}


def calculate_accuracies(conf_matrix):
    # Label 0 accuracy = True Negatives / (True Negatives + False Positives)
    accuracy_label_0 = conf_matrix[0][0] / (conf_matrix[0][0] + conf_matrix[0][1])
    # Label 1 accuracy = True Positives / (True Positives + False Negatives)
    accuracy_label_1 = conf_matrix[1][1] / (conf_matrix[1][0] + conf_matrix[1][1])
    return accuracy_label_0, accuracy_label_1


labels = list(confusion_matrices.keys())
accuracies_label_0 = []
accuracies_label_1 = []

for label in labels:
    conf_matrix = confusion_matrices[label]
    acc_label_0, acc_label_1 = calculate_accuracies(conf_matrix)
    accuracies_label_0.append(acc_label_0)
    accuracies_label_1.append(acc_label_1)


x = np.arange(len(labels))

fig, ax = plt.subplots(figsize=(12, 8))
width = 0.35


color_label_0 = 'skyblue'
color_label_1 = 'orange'


bars_label_0 = ax.bar(x - width/2, accuracies_label_0, width, label='Accuracy for Label 0', color=color_label_0)
bars_label_1 = ax.bar(x + width/2, accuracies_label_1, width, label='Accuracy for Label 1', color=color_label_1)


ax.set_xlabel('Model Configuration')
ax.set_ylabel('Accuracy')
ax.set_title('Accuracy for Label 0 and Label 1 in Different Models')
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=45, ha='right')
ax.legend()


plt.tight_layout()
plt.show()


# In[ ]:





# In[8]:


import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

def plot_confusion_matrices(y_tr, y_tr_pred, y_val, y_val_pred, y_test, y_test_pred):
    titles = ["Training Set", "Validation Set", "Test Set"]
    datasets = [(y_tr, y_tr_pred), (y_val, y_val_pred), (y_test, y_test_pred)]

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    for ax, (y_true, y_pred), title in zip(axes, datasets, titles):
        cm = confusion_matrix(y_true, y_pred)
        sns.heatmap(cm, annot=True, fmt="d", cmap="coolwarm", cbar=False, ax=ax)
        ax.set_title(f"Confusion Matrix - {title}")
        ax.set_xlabel("Predicted")
        ax.set_ylabel("Actual")

    plt.tight_layout()
    plt.show()
plot_confusion_matrices(y_train, y_train_pred, y_val, y_val_pred, y_test, y_test_pred)


# In[17]:


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Define confusion matrices
conf_matrices = {
    "class Weighting Training": np.array([[15481, 7319], [598, 1982]],),
    "class Weighting Validation": np.array( [[15263, 7033], [905, 1643]]),
    "class Weighting Test": np.array([[63815, 67], [7342, 13]])
}

# Plot each confusion matrix
plt.figure(figsize=(18, 5))
for i, (title, matrix) in enumerate(conf_matrices.items(), 1):
    plt.subplot(1, 3, i)
    sns.heatmap(matrix, annot=True, fmt='d', cmap='coolwarm', cbar=False,
                xticklabels=['Pred 0', 'Pred 1'], yticklabels=['True 0', 'True 1'])
    plt.title(title)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')

plt.tight_layout()
plt.suptitle("Confusion Matrices After Class weighting", fontsize=16, y=1.05)
plt.show()


# In[22]:


from sklearn.ensemble import GradientBoostingClassifier
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import accuracy_score, f1_score, recall_score, confusion_matrix, roc_curve
import numpy as np

sample_weights = compute_sample_weight(class_weight='balanced', y=y_train)

# Train the model with sample weights
gb_weighted = GradientBoostingClassifier(n_estimators=30, learning_rate=0.1, max_depth=3, random_state=4)
gb_weighted.fit(X_train, y_train, sample_weight=sample_weights)

# Predictions
y_train_pred = gb_weighted.predict(X_train)
y_val_pred = gb_weighted.predict(X_val)
y_test_pred = gb_weighted.predict(X_test)

# Evaluation function
def evaluate(y_true, y_pred, dataset_name):
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    fpr, tpr, _ = roc_curve(y_true, gb_weighted.predict_proba(X_val)[:,1])
    eer = fpr[np.nanargmin(np.absolute((1 - tpr) - fpr))]

    print(f"{dataset_name} Results:")
    print("Accuracy:", round(acc, 4))
    print("F1 Score:", round(f1, 4))
    print("Recall:", round(rec, 4))
    print("Confusion Matrix:\n", cm)
    print("Equal Error Rate (EER):", round(eer, 4))
    print()


# In[23]:


def evaluate(X_eval, y_true, y_pred, dataset_name):
    from sklearn.metrics import accuracy_score, f1_score, recall_score, confusion_matrix, roc_curve
    import numpy as np

    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)

    y_score = gb_weighted.predict_proba(X_eval)[:, 1]
    fpr, tpr, _ = roc_curve(y_true, y_score)
    eer = fpr[np.nanargmin(np.absolute((1 - tpr) - fpr))]

    print(f"{dataset_name} Results:")
    print("Accuracy:", round(acc, 4))
    print("F1 Score:", round(f1, 4))
    print("Recall:", round(rec, 4))
    print("Confusion Matrix:\n", cm)
    print("Equal Error Rate (EER):", round(eer, 4))
    print()

# Evaluate
evaluate(X_train, y_train, y_train_pred, "Training")
evaluate(X_val, y_val, y_val_pred, "Validation")
evaluate(X_test, y_test, y_test_pred, "Test")


# In[24]:


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import (
    confusion_matrix,
    roc_curve,
    precision_recall_curve
)


def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    threshold = thresholds[eer_index]
    return eer, fpr, tpr, threshold

def plot_graphs_only(model, X, y, set_name, row, fig):
    y_pred = model.predict(X)
    y_probs = model.predict_proba(X)[:, 1]

    cm = confusion_matrix(y, y_pred)
    eer, fpr, tpr, eer_threshold = compute_eer(y, y_probs)
    precision, recall, _ = precision_recall_curve(y, y_probs)

    # Plot 1: Confusion Matrix
    ax1 = fig.add_subplot(3, 3, 3*row + 1)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax1, cbar=False)
    ax1.set_title(f"{set_name} - Confusion Matrix")
    ax1.set_xlabel("Predicted")
    ax1.set_ylabel("Actual")

    # Plot 2: ROC Curve
    ax2 = fig.add_subplot(3, 3, 3*row + 2)
    ax2.plot(fpr, tpr, label="ROC Curve")
    ax2.plot([0, 1], [0, 1], 'k--', alpha=0.6)
    ax2.axvline(eer, color='red', linestyle='--', label=f"EER = {eer:.4f}")
    ax2.set_title(f"{set_name} - ROC Curve")
    ax2.set_xlabel("False Positive Rate")
    ax2.set_ylabel("True Positive Rate")
    ax2.legend()
    ax2.grid(True)

    # Plot 3: Precision-Recall Curve
    ax3 = fig.add_subplot(3, 3, 3*row + 3)
    ax3.plot(recall, precision, label="PR Curve")
    ax3.set_title(f"{set_name} - Precision-Recall Curve")
    ax3.set_xlabel("Recall")
    ax3.set_ylabel("Precision")
    ax3.legend()
    ax3.grid(True)



fig = plt.figure(figsize=(18, 12))
plot_graphs_only(gb_weighted, X_train, y_train, "Training (Oversampled)", 0, fig)
plot_graphs_only(gb_weighted, X_val, y_val, "Validation", 1, fig)
plot_graphs_only(gb_weighted, X_test, y_test, "Test", 2, fig)
plt.tight_layout()
plt.show()


# In[26]:


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def plot_all_graphs(model, X_sets, y_sets, set_names):
    num_sets = len(X_sets)
    fig, axes = plt.subplots(nrows=num_sets, ncols=3, figsize=(18, 4 * num_sets))

    for i, (X, y, name) in enumerate(zip(X_sets, y_sets, set_names)):
        y_pred = model.predict(X)
        y_probs = model.predict_proba(X)[:, 1]

        # 1. Actual Labels
        sns.countplot(x=y, hue=y, palette='cool', ax=axes[i, 0], legend=False)
        axes[i, 0].set_title(f'{name} - Actual Labels')
        axes[i, 0].set_xlabel('Class')
        axes[i, 0].set_ylabel('Count')

        # 2. Predicted Labels
        sns.countplot(x=y_pred, hue=y_pred, palette='magma', ax=axes[i, 1], legend=False)
        axes[i, 1].set_title(f'{name} - Predicted Labels')
        axes[i, 1].set_xlabel('Class')
        axes[i, 1].set_ylabel('Count')

        # 3. Scatter Plot
        sns.scatterplot(x=np.arange(len(y_probs)), y=y_probs, hue=y, palette='Set2', alpha=0.7, ax=axes[i, 2])
        axes[i, 2].set_title(f'{name} - Probabilities vs True Labels')
        axes[i, 2].set_xlabel('Sample Index')
        axes[i, 2].set_ylabel('Predicted Probability')
        axes[i, 2].legend(title='True Label')

    plt.tight_layout()
    plt.show()


plot_all_graphs(
    gb_weighted,
    [X_train, X_val, X_test],
    [y_train, y_val, y_test],
    ["Training", "Validation", "Test"]
)


# In[ ]:






## XG-Boost (Sailaja)

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, confusion_matrix, roc_curve, auc,precision_recall_curve


# In[2]:


train_df = pd.read_csv("train.csv")
val_df = pd.read_csv("dev.csv")
test_df = pd.read_csv("eval.csv")


# In[3]:


X_train = train_df.iloc[:, 2:].values
y_train = train_df["label"].values

X_val = val_df.iloc[:, 2:].values
y_val = val_df["label"].values

X_test = test_df.iloc[:, 2:].values
y_test = test_df["label"].values


# In[5]:


from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, confusion_matrix, roc_curve
import numpy as np

# Step 1: Define the model
xgb_model = XGBClassifier(
    n_estimators=100,
    max_depth=4,
    learning_rate=0.1,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)

# Step 2: Fit the model
xgb_model.fit(X_train, y_train)

# Step 3: Predictions
train_preds = xgb_model.predict(X_train)
val_preds = xgb_model.predict(X_val)
test_preds = xgb_model.predict(X_test)

# Step 4: Evaluation function
def evaluate_xgb(y_true, y_pred, X_data, dataset_name):
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)

    fpr, tpr, _ = roc_curve(y_true, xgb_model.predict_proba(X_data)[:, 1])
    eer = fpr[np.nanargmin(np.abs((1 - tpr) - fpr))]

    print(f"{dataset_name} Results:")
    print(f"Accuracy: {acc:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Recall: {rec:.4f}")
    print(f"Confusion Matrix:\n{cm}")
    print(f"Equal Error Rate (EER): {eer:.4f}\n")

# Step 5: Run evaluations
evaluate_xgb(y_train, train_preds, X_train, "Training")
evaluate_xgb(y_val, val_preds, X_val, "Validation")
evaluate_xgb(y_test, test_preds, X_test, "Test (Evaluation)")


# In[9]:


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import (
    confusion_matrix,
    roc_curve,
    precision_recall_curve
)


def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    threshold = thresholds[eer_index]
    return eer, fpr, tpr, threshold

def plot_graphs_only(model, X, y, set_name, row, fig):
    y_pred = model.predict(X)
    y_probs = model.predict_proba(X)[:, 1]

    cm = confusion_matrix(y, y_pred)
    eer, fpr, tpr, eer_threshold = compute_eer(y, y_probs)
    precision, recall, _ = precision_recall_curve(y, y_probs)

    # Plot 1: Confusion Matrix
    ax1 = fig.add_subplot(3, 3, 3*row + 1)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax1, cbar=False)
    ax1.set_title(f"{set_name} - Confusion Matrix")
    ax1.set_xlabel("Predicted")
    ax1.set_ylabel("Actual")

    # Plot 2: ROC Curve
    ax2 = fig.add_subplot(3, 3, 3*row + 2)
    ax2.plot(fpr, tpr, label="ROC Curve")
    ax2.plot([0, 1], [0, 1], 'k--', alpha=0.6)
    ax2.axvline(eer, color='red', linestyle='--', label=f"EER = {eer:.4f}")
    ax2.set_title(f"{set_name} - ROC Curve")
    ax2.set_xlabel("False Positive Rate")
    ax2.set_ylabel("True Positive Rate")
    ax2.legend()
    ax2.grid(True)

    # Plot 3: Precision-Recall Curve
    ax3 = fig.add_subplot(3, 3, 3*row + 3)
    ax3.plot(recall, precision, label="PR Curve")
    ax3.set_title(f"{set_name} - Precision-Recall Curve")
    ax3.set_xlabel("Recall")
    ax3.set_ylabel("Precision")
    ax3.legend()
    ax3.grid(True)



fig = plt.figure(figsize=(18, 12))
plot_graphs_only(xgb_model, X_train, y_train, "Training (Oversampled)", 0, fig)
plot_graphs_only(xgb_model, X_val, y_val, "Validation", 1, fig)
plot_graphs_only(xgb_model, X_test, y_test, "Test", 2, fig)
plt.tight_layout()
plt.show()


# In[11]:


##################UNDERSAMPLING
print(train_df['label'].value_counts())
class_0 = train_df[train_df['label'] == 0]
class_1 = train_df[train_df['label'] == 1]
class_0_undersampled = class_0.sample(n=len(class_1), random_state=42)
train_balanced_df = pd.concat([class_0_undersampled, class_1], axis=0)
train_balanced_df = train_balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)
print("Balanced Training Set:")
print(train_balanced_df['label'].value_counts())


# In[ ]:





# In[8]:


get_ipython().system('pip install --upgrade xgboost')


# In[12]:


from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, confusion_matrix, roc_curve
import numpy as np


X_train_undersampling = train_balanced_df.iloc[:, 2:].values
y_train_undersampling = train_balanced_df["label"].values


under_xg_boost = XGBClassifier(
    n_estimators=100,
    max_depth=4,
    learning_rate=0.1,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)

under_xg_boost.fit(X_train_undersampling, y_train_undersampling)


train_preds = under_xg_boost.predict(X_train_undersampling)
val_preds = under_xg_boost.predict(X_val)
test_preds = under_xg_boost.predict(X_test)


def evaluate_xgb_under(y_true, y_pred, X_data, dataset_name, model):
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)

    fpr, tpr, _ = roc_curve(y_true, model.predict_proba(X_data)[:, 1])
    eer = fpr[np.nanargmin(np.abs((1 - tpr) - fpr))]

    print(f"{dataset_name} Results:")
    print(f"Accuracy: {acc:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Recall: {rec:.4f}")
    print(f"Confusion Matrix:\n{cm}")
    print(f"Equal Error Rate (EER): {eer:.4f}\n")


evaluate_xgb_under(y_train_undersampling, train_preds, X_train_undersampling, "Training (Undersampled)", under_xg_boost)
evaluate_xgb_under(y_val, val_preds, X_val, "Validation", under_xg_boost)
evaluate_xgb_under(y_test, test_preds, X_test, "Test", under_xg_boost)


# In[13]:


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import (
    confusion_matrix,
    roc_curve,
    precision_recall_curve
)


def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    threshold = thresholds[eer_index]
    return eer, fpr, tpr, threshold

def plot_graphs_only(model, X, y, set_name, row, fig):
    y_pred = model.predict(X)
    y_probs = model.predict_proba(X)[:, 1]

    cm = confusion_matrix(y, y_pred)
    eer, fpr, tpr, eer_threshold = compute_eer(y, y_probs)
    precision, recall, _ = precision_recall_curve(y, y_probs)

    # Plot 1: Confusion Matrix
    ax1 = fig.add_subplot(3, 3, 3*row + 1)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax1, cbar=False)
    ax1.set_title(f"{set_name} - Confusion Matrix")
    ax1.set_xlabel("Predicted")
    ax1.set_ylabel("Actual")

    # Plot 2: ROC Curve
    ax2 = fig.add_subplot(3, 3, 3*row + 2)
    ax2.plot(fpr, tpr, label="ROC Curve")
    ax2.plot([0, 1], [0, 1], 'k--', alpha=0.6)
    ax2.axvline(eer, color='red', linestyle='--', label=f"EER = {eer:.4f}")
    ax2.set_title(f"{set_name} - ROC Curve")
    ax2.set_xlabel("False Positive Rate")
    ax2.set_ylabel("True Positive Rate")
    ax2.legend()
    ax2.grid(True)

    # Plot 3: Precision-Recall Curve
    ax3 = fig.add_subplot(3, 3, 3*row + 3)
    ax3.plot(recall, precision, label="PR Curve")
    ax3.set_title(f"{set_name} - Precision-Recall Curve")
    ax3.set_xlabel("Recall")
    ax3.set_ylabel("Precision")
    ax3.legend()
    ax3.grid(True)



fig = plt.figure(figsize=(18, 12))
plot_graphs_only(under_xg_boost, X_train_undersampling, y_train_undersampling, "Training (Oversampled)", 0, fig)
plot_graphs_only(under_xg_boost, X_val, y_val, "Validation", 1, fig)
plot_graphs_only(under_xg_boost, X_test, y_test, "Test", 2, fig)
plt.tight_layout()
plt.show()


# In[ ]:





# In[14]:


########################SMOTING

from imblearn.over_sampling import SMOTE
import pandas as pd

# Apply SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Convert to DataFrame (optional but useful for saving or inspecting)
OVERtrain_balanced_df = pd.DataFrame(X_train_resampled)
OVERtrain_balanced_df['label'] = y_train_resampled

# Check new label distribution
print(OVERtrain_balanced_df['label'].value_counts())


# In[15]:


from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

xgb_model_over = XGBClassifier(
     n_estimators=50,
    max_depth=4,
    learning_rate=0.1,
    random_state=42
)


xgb_model_over.fit(X_train_resampled, y_train_resampled)

# Make predictions
over_y_train_pred = xgb_model_over.predict(X_train_resampled)
over_y_val_pred = xgb_model_over.predict(X_val)
over_y_test_pred = xgb_model_over.predict(X_test)


# In[16]:


def evaluate(y_true, y_pred, X_data, dataset_name):
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    fpr, tpr, _ = roc_curve(y_true, xgb_model_over.predict_proba(X_data)[:, 1])
    eer = fpr[np.nanargmin(np.absolute((1 - tpr) - fpr))]

    print(f"{dataset_name} Results:")
    print(f"Accuracy: {acc:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Recall: {rec:.4f}")
    print(f"Confusion Matrix:\n {cm}")
    print(f"Equal Error Rate (EER): {eer:.4f}\n")

evaluate(y_train_resampled, over_y_train_pred, X_train_resampled, "Training")
evaluate(y_val, over_y_val_pred, X_val, "Validation")
evaluate(y_test, over_y_test_pred, X_test, "Test (Evaluation)")


# In[18]:


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import (
    confusion_matrix,
    roc_curve,
    precision_recall_curve
)


def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    threshold = thresholds[eer_index]
    return eer, fpr, tpr, threshold

def plot_graphs_only(model, X, y, set_name, row, fig):
    y_pred = model.predict(X)
    y_probs = model.predict_proba(X)[:, 1]

    cm = confusion_matrix(y, y_pred)
    eer, fpr, tpr, eer_threshold = compute_eer(y, y_probs)
    precision, recall, _ = precision_recall_curve(y, y_probs)

    # Plot 1: Confusion Matrix
    ax1 = fig.add_subplot(3, 3, 3*row + 1)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax1, cbar=False)
    ax1.set_title(f"{set_name} - Confusion Matrix")
    ax1.set_xlabel("Predicted")
    ax1.set_ylabel("Actual")

    # Plot 2: ROC Curve
    ax2 = fig.add_subplot(3, 3, 3*row + 2)
    ax2.plot(fpr, tpr, label="ROC Curve")
    ax2.plot([0, 1], [0, 1], 'k--', alpha=0.6)
    ax2.axvline(eer, color='red', linestyle='--', label=f"EER = {eer:.4f}")
    ax2.set_title(f"{set_name} - ROC Curve")
    ax2.set_xlabel("False Positive Rate")
    ax2.set_ylabel("True Positive Rate")
    ax2.legend()
    ax2.grid(True)

    # Plot 3: Precision-Recall Curve
    ax3 = fig.add_subplot(3, 3, 3*row + 3)
    ax3.plot(recall, precision, label="PR Curve")
    ax3.set_title(f"{set_name} - Precision-Recall Curve")
    ax3.set_xlabel("Recall")
    ax3.set_ylabel("Precision")
    ax3.legend()
    ax3.grid(True)



fig = plt.figure(figsize=(18, 12))
plot_graphs_only(xgb_model_over, X_train_resampled, y_train_resampled, "Training (Oversampled)", 0, fig)
plot_graphs_only(xgb_model_over, X_val, y_val, "Validation", 1, fig)
plot_graphs_only(xgb_model_over, X_test, y_test, "Test", 2, fig)
plt.tight_layout()
plt.show()


# In[19]:


from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)


pca = PCA(n_components=50)
X_train_pca = pca.fit_transform(X_train_scaled)
X_val_pca = pca.transform(X_val_scaled)
X_test_pca = pca.transform(X_test_scaled)


model_pca = XGBClassifier(
     n_estimators=100,
    max_depth=4,
    learning_rate=0.1,
    random_state=42
)

model_pca.fit(X_train_pca, y_train)


pca_train_preds= model_pca.predict(X_train_pca)
pca_val_preds = model_pca.predict(X_val_pca)
pca_test_preds = model_pca.predict(X_test_pca)


# In[20]:


from sklearn.metrics import accuracy_score, classification_report, f1_score, recall_score, confusion_matrix, roc_curve
import numpy as np

def evaluate_pca(y_true, y_pred, X_data, dataset_name):
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)


    fpr, tpr, _ = roc_curve(y_true, model_pca.predict_proba(X_data)[:, 1])
    eer = fpr[np.nanargmin(np.absolute((1 - tpr) - fpr))]

    print(f"{dataset_name} Results:")
    print(f"Accuracy: {acc:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Recall: {rec:.4f}")
    print(f"Confusion Matrix:\n {cm}")
    print(f"Equal Error Rate (EER): {eer:.4f}\n")
evaluate_pca(y_train, pca_train_preds, X_train_pca, "Training")
evaluate_pca(y_val, pca_val_preds, X_val_pca, "Validation")
evaluate_pca(y_test, pca_test_preds, X_test_pca, "Test (Evaluation)")


# In[22]:


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import (
    confusion_matrix,
    roc_curve,
    precision_recall_curve
)


def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    threshold = thresholds[eer_index]
    return eer, fpr, tpr, threshold

def plot_graphs_only(model, X, y, set_name, row, fig):
    y_pred = model.predict(X)
    y_probs = model.predict_proba(X)[:, 1]

    cm = confusion_matrix(y, y_pred)
    eer, fpr, tpr, eer_threshold = compute_eer(y, y_probs)
    precision, recall, _ = precision_recall_curve(y, y_probs)

    # Plot 1: Confusion Matrix
    ax1 = fig.add_subplot(3, 3, 3*row + 1)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax1, cbar=False)
    ax1.set_title(f"{set_name} - Confusion Matrix")
    ax1.set_xlabel("Predicted")
    ax1.set_ylabel("Actual")

    # Plot 2: ROC Curve
    ax2 = fig.add_subplot(3, 3, 3*row + 2)
    ax2.plot(fpr, tpr, label="ROC Curve")
    ax2.plot([0, 1], [0, 1], 'k--', alpha=0.6)
    ax2.axvline(eer, color='red', linestyle='--', label=f"EER = {eer:.4f}")
    ax2.set_title(f"{set_name} - ROC Curve")
    ax2.set_xlabel("False Positive Rate")
    ax2.set_ylabel("True Positive Rate")
    ax2.legend()
    ax2.grid(True)

    # Plot 3: Precision-Recall Curve
    ax3 = fig.add_subplot(3, 3, 3*row + 3)
    ax3.plot(recall, precision, label="PR Curve")
    ax3.set_title(f"{set_name} - Precision-Recall Curve")
    ax3.set_xlabel("Recall")
    ax3.set_ylabel("Precision")
    ax3.legend()
    ax3.grid(True)



fig = plt.figure(figsize=(18, 12))
plot_graphs_only(model_pca, X_train_pca, y_train, "Training ", 0, fig)
plot_graphs_only(model_pca, X_val_pca, y_val, "Validation", 1, fig)
plot_graphs_only(model_pca, X_test_pca, y_test, "Test", 2, fig)
plt.tight_layout()
plt.show()


# In[27]:


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def plot_all_graphs(model, X_sets, y_sets, set_names):
    num_sets = len(X_sets)
    fig, axes = plt.subplots(nrows=num_sets, ncols=3, figsize=(18, 4 * num_sets))

    for i, (X, y, name) in enumerate(zip(X_sets, y_sets, set_names)):
        y_pred = model.predict(X)
        y_probs = model.predict_proba(X)[:, 1]

        # 1. Actual Labels
        sns.countplot(x=y, hue=y, palette='cool', ax=axes[i, 0], legend=False)
        axes[i, 0].set_title(f'{name} - Actual Labels')
        axes[i, 0].set_xlabel('Class')
        axes[i, 0].set_ylabel('Count')

        # 2. Predicted Labels
        sns.countplot(x=y_pred, hue=y_pred, palette='magma', ax=axes[i, 1], legend=False)
        axes[i, 1].set_title(f'{name} - Predicted Labels')
        axes[i, 1].set_xlabel('Class')
        axes[i, 1].set_ylabel('Count')

        # 3. Scatter Plot
        sns.scatterplot(x=np.arange(len(y_probs)), y=y_probs, hue=y, palette='Set2', alpha=0.7, ax=axes[i, 2])
        axes[i, 2].set_title(f'{name} - Probabilities vs True Labels')
        axes[i, 2].set_xlabel('Sample Index')
        axes[i, 2].set_ylabel('Predicted Probability')
        axes[i, 2].legend(title='True Label')

    plt.tight_layout()
    plt.show()


# Example call
plot_all_graphs(
    xgb_model,
    [X_train, X_val, X_test],
    [y_train, y_val, y_test],
    ["Training", "Validation", "Test"]
)


# In[28]:


plot_all_graphs(
    under_xg_boost,
    [X_train_undersampling, X_val, X_test],
    [y_train_undersampling, y_val, y_test],
    ["Training", "Validation", "Test"]
)


# In[30]:


plot_all_graphs(
    xgb_model_over,
    [X_train_resampled, X_val, X_test],
    [y_train_resampled, y_val, y_test],
    ["Training", "Validation", "Test"]
)


# In[34]:


##########PCA
plot_all_graphs(
    model_pca,
    [X_train_pca, X_val_pca, X_test_pca],
    [y_train, y_val, y_test],
    ["Training", "Validation", "Test"]
)


# In[7]:


import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

def plot_confusion_matrices(y_tr, y_tr_pred, y_val, y_val_pred, y_test, y_test_pred):
    titles = ["Training Set", "Validation Set", "Test Set"]
    datasets = [(y_tr, y_tr_pred), (y_val, y_val_pred), (y_test, y_test_pred)]

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    for ax, (y_true, y_pred), title in zip(axes, datasets, titles):
        cm = confusion_matrix(y_true, y_pred)
        sns.heatmap(cm, annot=True, fmt="d", cmap="coolwarm", cbar=False, ax=ax)
        ax.set_title(f"Confusion Matrix - {title}")
        ax.set_xlabel("Predicted")
        ax.set_ylabel("Actual")

    plt.tight_layout()
    plt.show()


# In[8]:


plot_confusion_matrices(y_train, train_preds, y_val, val_preds, y_test, test_preds)


# In[40]:


plot_confusion_matrices(y_train_resampled, over_y_train_pred, y_val, over_y_val_pred, y_test, over_y_test_pred)


# In[41]:



plot_confusion_matrices(y_train_undersampling, under_y_train_pred, y_val, under_y_val_pred, y_test, under_y_test_pred)


# In[42]:


plot_confusion_matrices(y_train, pca_train_preds, y_val, pca_val_preds, y_test, pca_test_preds)


# In[1]:


import matplotlib.pyplot as plt
import numpy as np

confusion_matrices = {
    "Original Training": [[22798, 2], [2391, 189]],
    "Original Validation": [[22287, 9], [2532, 16]],
    "Original Test": [[63882, 0], [7355, 0]],

    "Undersampling Training": [[2343, 237], [140, 2440]],
    "Undersampling Validation": [[15472, 6824], [745, 1803]],
    "Undersampling Test": [[63375, 507], [7226, 129]],

    "Oversampling Training": [[19174, 3626], [1519, 21281]],
    "Oversampling Validation": [[18160, 4136], [1257, 1291]],
    "Oversampling Test": [[63882, 0], [7355, 0]],

    "PCA Training": [[22800, 0], [2563, 17]],
    "PCA Validation": [[22296, 0], [2548, 0]],
    "PCA Test": [[63882, 0], [7355, 0]]
}


# Function to calculate accuracy for label 0 and label 1
def calculate_accuracies(conf_matrix):
    # Label 0 accuracy = True Negatives / (True Negatives + False Positives)
    accuracy_label_0 = conf_matrix[0][0] / (conf_matrix[0][0] + conf_matrix[0][1])
    # Label 1 accuracy = True Positives / (True Positives + False Negatives)
    accuracy_label_1 = conf_matrix[1][1] / (conf_matrix[1][0] + conf_matrix[1][1])
    return accuracy_label_0, accuracy_label_1

# Prepare data for plotting
labels = list(confusion_matrices.keys())
accuracies_label_0 = []
accuracies_label_1 = []

for label in labels:
    conf_matrix = confusion_matrices[label]
    acc_label_0, acc_label_1 = calculate_accuracies(conf_matrix)
    accuracies_label_0.append(acc_label_0)
    accuracies_label_1.append(acc_label_1)


x = np.arange(len(labels))  # Label positions

fig, ax = plt.subplots(figsize=(12, 8))
width = 0.35  # Bar width


color_label_0 = 'skyblue'
color_label_1 = 'orange'

# Plotting bars for Label 0 and Label 1
bars_label_0 = ax.bar(x - width/2, accuracies_label_0, width, label='Accuracy for Label 0', color=color_label_0)
bars_label_1 = ax.bar(x + width/2, accuracies_label_1, width, label='Accuracy for Label 1', color=color_label_1)

# Labeling and formatting
ax.set_xlabel('Model Configuration')
ax.set_ylabel('Accuracy')
ax.set_title('Accuracy for Label 0 and Label 1 in Different Models')
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=45, ha='right')
ax.legend()


plt.tight_layout()
plt.show()


# In[ ]:






## Logistic Classification (Sailaja)

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[3]:


import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix,recall_score, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE


# In[4]:


train_df = pd.read_csv("train.csv")
val_df = pd.read_csv("dev.csv")
test_df = pd.read_csv("eval.csv")


# In[5]:


X_train = train_df.iloc[:, 2:].values
y_train = train_df["label"].values

X_val = val_df.iloc[:, 2:].values
y_val = val_df["label"].values

X_test = test_df.iloc[:, 2:].values
y_test = test_df["label"].values


# In[7]:


#######VALIDATION SET
log_reg = LogisticRegression(max_iter=500)
log_reg.fit(X_train, y_train)


y_train_pred = log_reg.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
train_recall = recall_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred)
y_probs_train = log_reg.predict_proba(X_train)[:, 1]

def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    return eer

print("\nTraining Results:")
print(f"Accuracy: {train_accuracy:.4f}")
print(f"F1 Score: {train_f1:.4f}")
print(f"Recall Score: {train_recall:.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_train, y_train_pred))
print(f"Equal Error Rate: {compute_eer(y_train, y_probs_train):.4f}")


# In[8]:


y_val_pred = log_reg.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
val_recall=recall_score(y_val,y_val_pred)
val_f1 = f1_score(y_val, y_val_pred)
y_probs = log_reg.predict_proba(X_val)[:, 1]

print("Validation Results:")
print(f"Accuracy: {val_accuracy:.4f}")
print(f"F1 Score: {val_f1:.4f}")
print(f"Recall Score: {val_recall:.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_val_pred))
eer = compute_eer(y_val, y_probs)
print(f"Equal Error Rate: {eer:.4f}")


# In[9]:


###TEST SET
y_test_pred = log_reg.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)
test_recall=recall_score(y_test,y_test_pred)
y_probs_test = log_reg.predict_proba(X_test)[:, 1]
def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    return eer

print("\nTest (Evaluation) Results:")
print(f"Accuracy: {test_accuracy:.4f}")
print(f"F1 Score: {test_f1:.4f}")
print(f"Recall Score: {test_recall:.4f}")
eer_test = compute_eer(y_test, y_probs_test)
print(f"Equal Error Rate: {eer_test:.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))


# In[10]:


plt.figure(figsize=(12, 5))

# Confusion Matrix - Validation Set
plt.subplot(1, 2, 1)
sns.heatmap(confusion_matrix(y_val, y_val_pred), annot=True, fmt="d", cmap="coolwarm", xticklabels=["Class 0", "Class 1"], yticklabels=["Class 0", "Class 1"])
plt.title("Confusion Matrix (Validation Set)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")


plt.subplot(1, 2, 2)
sns.heatmap(confusion_matrix(y_test, y_test_pred),  annot=True, fmt="d", cmap="coolwarm", xticklabels=["Class 0", "Class 1"], yticklabels=["Class 0", "Class 1"])
plt.title("Confusion Matrix (Test Set)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")

plt.tight_layout()
plt.show()


# In[12]:


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import (
    confusion_matrix,
    roc_curve,
    precision_recall_curve
)


def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    threshold = thresholds[eer_index]
    return eer, fpr, tpr, threshold

def plot_graphs_only(model, X, y, set_name, row, fig):
    y_pred = model.predict(X)
    y_probs = model.predict_proba(X)[:, 1]

    cm = confusion_matrix(y, y_pred)
    eer, fpr, tpr, eer_threshold = compute_eer(y, y_probs)
    precision, recall, _ = precision_recall_curve(y, y_probs)

    # Plot 1: Confusion Matrix
    ax1 = fig.add_subplot(3, 3, 3*row + 1)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax1, cbar=False)
    ax1.set_title(f"{set_name} - Confusion Matrix")
    ax1.set_xlabel("Predicted")
    ax1.set_ylabel("Actual")

    # Plot 2: ROC Curve
    ax2 = fig.add_subplot(3, 3, 3*row + 2)
    ax2.plot(fpr, tpr, label="ROC Curve")
    ax2.plot([0, 1], [0, 1], 'k--', alpha=0.6)
    ax2.axvline(eer, color='red', linestyle='--', label=f"EER = {eer:.4f}")
    ax2.set_title(f"{set_name} - ROC Curve")
    ax2.set_xlabel("False Positive Rate")
    ax2.set_ylabel("True Positive Rate")
    ax2.legend()
    ax2.grid(True)

    # Plot 3: Precision-Recall Curve
    ax3 = fig.add_subplot(3, 3, 3*row + 3)
    ax3.plot(recall, precision, label="PR Curve")
    ax3.set_title(f"{set_name} - Precision-Recall Curve")
    ax3.set_xlabel("Recall")
    ax3.set_ylabel("Precision")
    ax3.legend()
    ax3.grid(True)



fig = plt.figure(figsize=(18, 12))
plot_graphs_only(log_reg, X_train, y_train, "Training (Oversampled)", 0, fig)
plot_graphs_only(log_reg, X_val, y_val, "Validation", 1, fig)
plot_graphs_only(log_reg, X_test, y_test, "Test", 2, fig)
plt.tight_layout()
plt.show()


# In[29]:


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def plot_all_graphs(model, X_sets, y_sets, set_names):
    num_sets = len(X_sets)
    fig, axes = plt.subplots(nrows=num_sets, ncols=3, figsize=(18, 4 * num_sets))

    for i, (X, y, name) in enumerate(zip(X_sets, y_sets, set_names)):
        y_pred = model.predict(X)
        y_probs = model.predict_proba(X)[:, 1]

        # 1. Actual Labels
        sns.countplot(x=y, hue=y, palette='cool', ax=axes[i, 0], legend=False)
        axes[i, 0].set_title(f'{name} - Actual Labels')
        axes[i, 0].set_xlabel('Class')
        axes[i, 0].set_ylabel('Count')

        # 2. Predicted Labels
        sns.countplot(x=y_pred, hue=y_pred, palette='magma', ax=axes[i, 1], legend=False)
        axes[i, 1].set_title(f'{name} - Predicted Labels')
        axes[i, 1].set_xlabel('Class')
        axes[i, 1].set_ylabel('Count')

        # 3. Scatter Plot
        sns.scatterplot(x=np.arange(len(y_probs)), y=y_probs, hue=y, palette='Set2', alpha=0.7, ax=axes[i, 2])
        axes[i, 2].set_title(f'{name} - Probabilities vs True Labels')
        axes[i, 2].set_xlabel('Sample Index')
        axes[i, 2].set_ylabel('Predicted Probability')
        axes[i, 2].legend(title='True Label')

    plt.tight_layout()
    plt.show()


# Example call
plot_all_graphs(
    log_reg,
    [X_train, X_val, X_test],
    [y_train, y_val, y_test],
    ["Training", "Validation", "Test"]
)


# In[13]:


#####UNDERSAMPLING
print(train_df['label'].value_counts())
class_0 = train_df[train_df['label'] == 0]
class_1 = train_df[train_df['label'] == 1]
class_0_undersampled = class_0.sample(n=len(class_1), random_state=42)
train_balanced_df = pd.concat([class_0_undersampled, class_1], axis=0)
train_balanced_df = train_balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)
print("Balanced Training Set:")
print(train_balanced_df['label'].value_counts())


# In[14]:


X_train_undersampling = train_balanced_df.iloc[:, 2:].values
y_train_undersampling = train_balanced_df["label"].values
UNDER_logistic = LogisticRegression(max_iter=500)
UNDER_logistic.fit(X_train_undersampling, y_train_undersampling)


# In[26]:


# Predict on training set
Under_y_train_pred = UNDER_logistic.predict(X_train_undersampling)
Under_train_accuracy = accuracy_score(y_train_undersampling, Under_y_train_pred)
Under_train_recall = recall_score(y_train_undersampling, Under_y_train_pred)
Under_train_f1 = f1_score(y_train_undersampling, Under_y_train_pred)
Under_y_train_probs = UNDER_logistic.predict_proba(X_train_undersampling)[:, 1]

# EER function
def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    return eer

# Print results
print("Training Results after Undersampling:")
print(f"Accuracy: {Under_train_accuracy:.4f}")
print(f"F1 Score: {Under_train_f1:.4f}")
print(f"Recall Score: {Under_train_recall:.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_train_undersampling, Under_y_train_pred))
print(f"Equal Error Rate: {compute_eer(y_train_undersampling, Under_y_train_probs):.4f}")


# In[16]:


#######VALIDATION SET AFTER UNDERSAMPLING
UNDERy_val_pred = UNDER_logistic.predict(X_val)
UNDERval_accuracy = accuracy_score(y_val, UNDERy_val_pred)
UNDERval_recall=recall_score(y_val,UNDERy_val_pred)
UNDERval_f1 = f1_score(y_val, UNDERy_val_pred)
UNDERy_probs = UNDER_logistic.predict_proba(X_val)[:, 1]
def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    return eer
print("Validation Results:")
print(f"Accuracy: {UNDERval_accuracy :.4f}")
print(f"F1 Score: {UNDERval_f1:.4f}")
print(f"Recall Score: {UNDERval_recall:.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_val, UNDERy_val_pred))
UNDEReer = compute_eer(y_val, y_probs)
print(f"Equal Error Rate: {UNDEReer:.4f}")


# In[17]:


###TEST SET AFTER UNDERSAMPLING
UNDERy_test_pred = UNDER_logistic.predict(X_test)
UNDERtest_accuracy = accuracy_score(y_test, UNDERy_test_pred)
UNDERtest_f1 = f1_score(y_test, UNDERy_test_pred)
UNDERtest_recall=recall_score(y_test,UNDERy_test_pred)
UNDERy_probs_test = UNDER_logistic.predict_proba(X_test)[:, 1]
def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    return eer

print("\nTest (Evaluation) Results:")
print(f"Accuracy: {UNDERtest_accuracy:.4f}")
print(f"F1 Score: {UNDERtest_f1:.4f}")
print(f"Recall Score: {UNDERtest_recall:.4f}")
UNDEReer_test = compute_eer(y_test, UNDERy_probs_test)
print(f"Equal Error Rate: {UNDEReer_test:.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, UNDERy_test_pred))


# In[18]:


plt.figure(figsize=(12, 5))

# Confusion Matrix - Validation Set
plt.subplot(1, 2, 1)
sns.heatmap(confusion_matrix(y_val, UNDERy_val_pred), annot=True, fmt="d", cmap="coolwarm", xticklabels=["Class 0", "Class 1"], yticklabels=["Class 0", "Class 1"])
plt.title("Confusion Matrix (Validation Set)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")


plt.subplot(1, 2, 2)
sns.heatmap(confusion_matrix(y_test, UNDERy_test_pred),  annot=True, fmt="d", cmap="coolwarm", xticklabels=["Class 0", "Class 1"], yticklabels=["Class 0", "Class 1"])
plt.title("Confusion Matrix (Test Set)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")

plt.tight_layout()
plt.show()


# In[19]:


####OVERSAMPLING

print("Original Class Distribution:")
print(train_df['label'].value_counts())

class_0 = train_df[train_df['label'] == 0]
class_1 = train_df[train_df['label'] == 1]

class_1_oversampled = class_1.sample(n=len(class_0), replace=True, random_state=42)

OVERtrain_balanced_df = pd.concat([class_0, class_1_oversampled], axis=0)

# Shuffle the combined dataset
OVERtrain_balanced_df = OVERtrain_balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Show the new class distribution
print("Balanced Training Set After Oversampling:")
print(OVERtrain_balanced_df['label'].value_counts())


# In[20]:


X_train_oversampling = OVERtrain_balanced_df.iloc[:, 2:].values
y_train_oversampling = OVERtrain_balanced_df["label"].values
OVER_logistic = LogisticRegression(max_iter=500)
OVER_logistic.fit(X_train_oversampling, y_train_oversampling)


# In[21]:


# Predict on training set
over_y_train_pred = OVER_logistic.predict(X_train_oversampling)
over_train_accuracy = accuracy_score(y_train_oversampling, over_y_train_pred)
over_train_recall = recall_score(y_train_oversampling, over_y_train_pred)
over_train_f1 = f1_score(y_train_oversampling, over_y_train_pred)
over_y_train_probs = OVER_logistic.predict_proba(X_train_oversampling)[:, 1]

# EER function
def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    return eer

# Print results
print("Training Results after Undersampling:")
print(f"Accuracy: {over_train_accuracy:.4f}")
print(f"F1 Score: {over_train_f1:.4f}")
print(f"Recall Score: {over_train_recall:.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_train_oversampling, over_y_train_pred))
print(f"Equal Error Rate: {compute_eer(y_train_oversampling, over_y_train_probs):.4f}")


# In[22]:


######VALIDATION METRICS AFTER OVERSAMPLING
OVERy_val_pred = OVER_logistic.predict(X_val)
OVERval_accuracy = accuracy_score(y_val, OVERy_val_pred)
OVERval_recall=recall_score(y_val,OVERy_val_pred)
OVERval_f1 = f1_score(y_val, OVERy_val_pred)
OVERy_probs = OVER_logistic.predict_proba(X_val)[:, 1]
def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    return eer
print("Validation Results:")
print(f"Accuracy: {OVERval_accuracy :.4f}")
print(f"F1 Score: {OVERval_f1:.4f}")
print(f"Recall Score: {OVERval_recall:.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_val, OVERy_val_pred))
OVEReer = compute_eer(y_val, OVERy_probs)
print(f"Equal Error Rate: {OVEReer:.4f}")


# In[23]:


###TEST SET AFTER OVERSAMPLING
OVERy_test_pred = OVER_logistic.predict(X_test)
OVERtest_accuracy = accuracy_score(y_test, OVERy_test_pred)
OVERtest_f1 = f1_score(y_test,OVERy_test_pred)
OVERtest_recall=recall_score(y_test,OVERy_test_pred)
OVERy_probs_test = OVER_logistic.predict_proba(X_test)[:, 1]
def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    return eer

print("\nTest (Evaluation) Results:")
print(f"Accuracy: {OVERtest_accuracy:.4f}")
print(f"F1 Score: {OVERtest_f1:.4f}")
print(f"Recall Score: {OVERtest_recall:.4f}")
OVEReer_test = compute_eer(y_test, OVERy_probs_test)
print(f"Equal Error Rate: {OVEReer_test:.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, OVERy_test_pred))


# In[24]:


plt.figure(figsize=(12, 5))


plt.subplot(1, 2, 1)
sns.heatmap(confusion_matrix(y_val, OVERy_val_pred), annot=True, fmt="d", cmap="coolwarm", xticklabels=["Class 0", "Class 1"], yticklabels=["Class 0", "Class 1"])
plt.title("Confusion Matrix (Validation Set)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")


plt.subplot(1, 2, 2)
sns.heatmap(confusion_matrix(y_test,OVERy_test_pred),  annot=True, fmt="d", cmap="coolwarm", xticklabels=["Class 0", "Class 1"], yticklabels=["Class 0", "Class 1"])
plt.title("Confusion Matrix (Test Set)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")

plt.tight_layout()
plt.show()


# In[ ]:





# In[ ]:





# In[11]:


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import (
    confusion_matrix,
    roc_curve,
    precision_recall_curve
)


def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    threshold = thresholds[eer_index]
    return eer, fpr, tpr, threshold

def plot_graphs_only(model, X, y, set_name, row, fig):
    y_pred = model.predict(X)
    y_probs = model.predict_proba(X)[:, 1]

    cm = confusion_matrix(y, y_pred)
    eer, fpr, tpr, eer_threshold = compute_eer(y, y_probs)
    precision, recall, _ = precision_recall_curve(y, y_probs)

    # Plot 1: Confusion Matrix
    ax1 = fig.add_subplot(3, 3, 3*row + 1)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax1, cbar=False)
    ax1.set_title(f"{set_name} - Confusion Matrix")
    ax1.set_xlabel("Predicted")
    ax1.set_ylabel("Actual")

    # Plot 2: ROC Curve
    ax2 = fig.add_subplot(3, 3, 3*row + 2)
    ax2.plot(fpr, tpr, label="ROC Curve")
    ax2.plot([0, 1], [0, 1], 'k--', alpha=0.6)
    ax2.axvline(eer, color='red', linestyle='--', label=f"EER = {eer:.4f}")
    ax2.set_title(f"{set_name} - ROC Curve")
    ax2.set_xlabel("False Positive Rate")
    ax2.set_ylabel("True Positive Rate")
    ax2.legend()
    ax2.grid(True)

    # Plot 3: Precision-Recall Curve
    ax3 = fig.add_subplot(3, 3, 3*row + 3)
    ax3.plot(recall, precision, label="PR Curve")
    ax3.set_title(f"{set_name} - Precision-Recall Curve")
    ax3.set_xlabel("Recall")
    ax3.set_ylabel("Precision")
    ax3.legend()
    ax3.grid(True)



fig = plt.figure(figsize=(18, 12))
plot_graphs_only(log_reg, X_train, y_train, "Training (Oversampled)", 0, fig)
plot_graphs_only(log_reg, X_val, y_val, "Validation", 1, fig)
plot_graphs_only(log_reg, X_test, y_test, "Test", 2, fig)
plt.tight_layout()
plt.show()


# In[27]:


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import (
    confusion_matrix,
    roc_curve,
    precision_recall_curve
)


def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    threshold = thresholds[eer_index]
    return eer, fpr, tpr, threshold

def plot_graphs_only(model, X, y, set_name, row, fig):
    y_pred = model.predict(X)
    y_probs = model.predict_proba(X)[:, 1]

    cm = confusion_matrix(y, y_pred)
    eer, fpr, tpr, eer_threshold = compute_eer(y, y_probs)
    precision, recall, _ = precision_recall_curve(y, y_probs)

    # Plot 1: Confusion Matrix
    ax1 = fig.add_subplot(3, 3, 3*row + 1)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax1, cbar=False)
    ax1.set_title(f"{set_name} - Confusion Matrix")
    ax1.set_xlabel("Predicted")
    ax1.set_ylabel("Actual")

    # Plot 2: ROC Curve
    ax2 = fig.add_subplot(3, 3, 3*row + 2)
    ax2.plot(fpr, tpr, label="ROC Curve")
    ax2.plot([0, 1], [0, 1], 'k--', alpha=0.6)
    ax2.axvline(eer, color='red', linestyle='--', label=f"EER = {eer:.4f}")
    ax2.set_title(f"{set_name} - ROC Curve")
    ax2.set_xlabel("False Positive Rate")
    ax2.set_ylabel("True Positive Rate")
    ax2.legend()
    ax2.grid(True)

    # Plot 3: Precision-Recall Curve
    ax3 = fig.add_subplot(3, 3, 3*row + 3)
    ax3.plot(recall, precision, label="PR Curve")
    ax3.set_title(f"{set_name} - Precision-Recall Curve")
    ax3.set_xlabel("Recall")
    ax3.set_ylabel("Precision")
    ax3.legend()
    ax3.grid(True)



fig = plt.figure(figsize=(18, 12))
plot_graphs_only(UNDER_logistic, X_train_undersampling, y_train_undersampling, "Training (Oversampled)", 0, fig)
plot_graphs_only(UNDER_logistic, X_val, y_val, "Validation", 1, fig)
plot_graphs_only(UNDER_logistic, X_test, y_test, "Test", 2, fig)
plt.tight_layout()
plt.show()


# In[28]:


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import (
    confusion_matrix,
    roc_curve,
    precision_recall_curve
)


def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    threshold = thresholds[eer_index]
    return eer, fpr, tpr, threshold

def plot_graphs_only(model, X, y, set_name, row, fig):
    y_pred = model.predict(X)
    y_probs = model.predict_proba(X)[:, 1]

    cm = confusion_matrix(y, y_pred)
    eer, fpr, tpr, eer_threshold = compute_eer(y, y_probs)
    precision, recall, _ = precision_recall_curve(y, y_probs)

    # Plot 1: Confusion Matrix
    ax1 = fig.add_subplot(3, 3, 3*row + 1)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax1, cbar=False)
    ax1.set_title(f"{set_name} - Confusion Matrix")
    ax1.set_xlabel("Predicted")
    ax1.set_ylabel("Actual")

    # Plot 2: ROC Curve
    ax2 = fig.add_subplot(3, 3, 3*row + 2)
    ax2.plot(fpr, tpr, label="ROC Curve")
    ax2.plot([0, 1], [0, 1], 'k--', alpha=0.6)
    ax2.axvline(eer, color='red', linestyle='--', label=f"EER = {eer:.4f}")
    ax2.set_title(f"{set_name} - ROC Curve")
    ax2.set_xlabel("False Positive Rate")
    ax2.set_ylabel("True Positive Rate")
    ax2.legend()
    ax2.grid(True)

    # Plot 3: Precision-Recall Curve
    ax3 = fig.add_subplot(3, 3, 3*row + 3)
    ax3.plot(recall, precision, label="PR Curve")
    ax3.set_title(f"{set_name} - Precision-Recall Curve")
    ax3.set_xlabel("Recall")
    ax3.set_ylabel("Precision")
    ax3.legend()
    ax3.grid(True)



fig = plt.figure(figsize=(18, 12))
plot_graphs_only(OVER_logistic, X_train_oversampling, y_train_oversampling, "Training (Oversampled)", 0, fig)
plot_graphs_only(OVER_logistic, X_val, y_val, "Validation", 1, fig)
plot_graphs_only(OVER_logistic, X_test, y_test, "Test", 2, fig)
plt.tight_layout()
plt.show()


# In[30]:


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def plot_all_graphs(model, X_sets, y_sets, set_names):
    num_sets = len(X_sets)
    fig, axes = plt.subplots(nrows=num_sets, ncols=3, figsize=(18, 4 * num_sets))

    for i, (X, y, name) in enumerate(zip(X_sets, y_sets, set_names)):
        y_pred = model.predict(X)
        y_probs = model.predict_proba(X)[:, 1]

        # 1. Actual Labels
        sns.countplot(x=y, hue=y, palette='cool', ax=axes[i, 0], legend=False)
        axes[i, 0].set_title(f'{name} - Actual Labels')
        axes[i, 0].set_xlabel('Class')
        axes[i, 0].set_ylabel('Count')

        # 2. Predicted Labels
        sns.countplot(x=y_pred, hue=y_pred, palette='magma', ax=axes[i, 1], legend=False)
        axes[i, 1].set_title(f'{name} - Predicted Labels')
        axes[i, 1].set_xlabel('Class')
        axes[i, 1].set_ylabel('Count')

        # 3. Scatter Plot
        sns.scatterplot(x=np.arange(len(y_probs)), y=y_probs, hue=y, palette='Set2', alpha=0.7, ax=axes[i, 2])
        axes[i, 2].set_title(f'{name} - Probabilities vs True Labels')
        axes[i, 2].set_xlabel('Sample Index')
        axes[i, 2].set_ylabel('Predicted Probability')
        axes[i, 2].legend(title='True Label')

    plt.tight_layout()
    plt.show()


# Example call
plot_all_graphs(
    UNDER_logistic,
    [X_train_undersampling, X_val, X_test],
    [y_train_undersampling, y_val, y_test],
    ["Training", "Validation", "Test"]
)


# In[31]:


plt.figure(figsize=(12, 5))

# Confusion Matrix - Validation Set
plt.subplot(1, 2, 1)
sns.heatmap(confusion_matrix(y_val, UNDERy_val_pred), annot=True, fmt="d", cmap="coolwarm", xticklabels=["Class 0", "Class 1"], yticklabels=["Class 0", "Class 1"])
plt.title("Confusion Matrix (Validation Set)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")


plt.subplot(1, 2, 2)
sns.heatmap(confusion_matrix(y_test, UNDERy_test_pred),  annot=True, fmt="d", cmap="coolwarm", xticklabels=["Class 0", "Class 1"], yticklabels=["Class 0", "Class 1"])
plt.title("Confusion Matrix (Test Set)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")

plt.tight_layout()
plt.show()


# In[32]:





# In[33]:


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def plot_all_graphs(model, X_sets, y_sets, set_names):
    num_sets = len(X_sets)
    fig, axes = plt.subplots(nrows=num_sets, ncols=3, figsize=(18, 4 * num_sets))

    for i, (X, y, name) in enumerate(zip(X_sets, y_sets, set_names)):
        y_pred = model.predict(X)
        y_probs = model.predict_proba(X)[:, 1]

        # 1. Actual Labels
        sns.countplot(x=y, hue=y, palette='cool', ax=axes[i, 0], legend=False)
        axes[i, 0].set_title(f'{name} - Actual Labels')
        axes[i, 0].set_xlabel('Class')
        axes[i, 0].set_ylabel('Count')

        # 2. Predicted Labels
        sns.countplot(x=y_pred, hue=y_pred, palette='magma', ax=axes[i, 1], legend=False)
        axes[i, 1].set_title(f'{name} - Predicted Labels')
        axes[i, 1].set_xlabel('Class')
        axes[i, 1].set_ylabel('Count')

        # 3. Scatter Plot
        sns.scatterplot(x=np.arange(len(y_probs)), y=y_probs, hue=y, palette='Set2', alpha=0.7, ax=axes[i, 2])
        axes[i, 2].set_title(f'{name} - Probabilities vs True Labels')
        axes[i, 2].set_xlabel('Sample Index')
        axes[i, 2].set_ylabel('Predicted Probability')
        axes[i, 2].legend(title='True Label')

    plt.tight_layout()
    plt.show()


# Example call
plot_all_graphs(
    OVER_logistic,
    [X_train_oversampling, X_val, X_test],
    [y_train_oversampling, y_val, y_test],
    ["Training", "Validation", "Test"]
)


# In[36]:


from sklearn.ensemble import GradientBoostingClassifier
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import accuracy_score, f1_score, recall_score, confusion_matrix, roc_curve
import numpy as np

# sample_weights = compute_sample_weight(class_weight='balanced', y=y_train)

# # Train the model with sample weights
# gb_weighted = GradientBoostingClassifier(n_estimators=30, learning_rate=0.1, max_depth=3, random_state=4)
# gb_weighted.fit(X_train, y_train, sample_weight=sample_weights)

from sklearn.linear_model import LogisticRegression

# Train Logistic Regression with balanced class weights
log_reg_balanced = LogisticRegression(class_weight='balanced', random_state=4, max_iter=1000)
log_reg_balanced.fit(X_train, y_train)

# Predictions
y_train_pred = log_reg_balanced.predict(X_train)
y_val_pred = log_reg_balanced.predict(X_val)
y_test_pred = log_reg_balanced.predict(X_test)

# Evaluation function
def evaluate(y_true, y_pred, dataset_name):
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    fpr, tpr, _ = roc_curve(y_true, gb_weighted.predict_proba(X_val)[:,1])
    eer = fpr[np.nanargmin(np.absolute((1 - tpr) - fpr))]

    print(f"{dataset_name} Results:")
    print("Accuracy:", round(acc, 4))
    print("F1 Score:", round(f1, 4))
    print("Recall:", round(rec, 4))
    print("Confusion Matrix:\n", cm)
    print("Equal Error Rate (EER):", round(eer, 4))
    print()


# In[37]:


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def plot_all_graphs(model, X_sets, y_sets, set_names):
    num_sets = len(X_sets)
    fig, axes = plt.subplots(nrows=num_sets, ncols=3, figsize=(18, 4 * num_sets))

    for i, (X, y, name) in enumerate(zip(X_sets, y_sets, set_names)):
        y_pred = model.predict(X)
        y_probs = model.predict_proba(X)[:, 1]

        # 1. Actual Labels
        sns.countplot(x=y, hue=y, palette='cool', ax=axes[i, 0], legend=False)
        axes[i, 0].set_title(f'{name} - Actual Labels')
        axes[i, 0].set_xlabel('Class')
        axes[i, 0].set_ylabel('Count')

        # 2. Predicted Labels
        sns.countplot(x=y_pred, hue=y_pred, palette='magma', ax=axes[i, 1], legend=False)
        axes[i, 1].set_title(f'{name} - Predicted Labels')
        axes[i, 1].set_xlabel('Class')
        axes[i, 1].set_ylabel('Count')

        # 3. Scatter Plot
        sns.scatterplot(x=np.arange(len(y_probs)), y=y_probs, hue=y, palette='Set2', alpha=0.7, ax=axes[i, 2])
        axes[i, 2].set_title(f'{name} - Probabilities vs True Labels')
        axes[i, 2].set_xlabel('Sample Index')
        axes[i, 2].set_ylabel('Predicted Probability')
        axes[i, 2].legend(title='True Label')

    plt.tight_layout()
    plt.show()


plot_all_graphs(
    log_reg_balanced,
    [X_train, X_val, X_test],
    [y_train, y_val, y_test],
    ["Training", "Validation", "Test"]
)


# In[39]:


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import (
    confusion_matrix,
    roc_curve,
    precision_recall_curve
)


def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    threshold = thresholds[eer_index]
    return eer, fpr, tpr, threshold

def plot_graphs_only(model, X, y, set_name, row, fig):
    y_pred = model.predict(X)
    y_probs = model.predict_proba(X)[:, 1]

    cm = confusion_matrix(y, y_pred)
    eer, fpr, tpr, eer_threshold = compute_eer(y, y_probs)
    precision, recall, _ = precision_recall_curve(y, y_probs)

    # Plot 1: Confusion Matrix
    ax1 = fig.add_subplot(3, 3, 3*row + 1)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax1, cbar=False)
    ax1.set_title(f"{set_name} - Confusion Matrix")
    ax1.set_xlabel("Predicted")
    ax1.set_ylabel("Actual")

    # Plot 2: ROC Curve
    ax2 = fig.add_subplot(3, 3, 3*row + 2)
    ax2.plot(fpr, tpr, label="ROC Curve")
    ax2.plot([0, 1], [0, 1], 'k--', alpha=0.6)
    ax2.axvline(eer, color='red', linestyle='--', label=f"EER = {eer:.4f}")
    ax2.set_title(f"{set_name} - ROC Curve")
    ax2.set_xlabel("False Positive Rate")
    ax2.set_ylabel("True Positive Rate")
    ax2.legend()
    ax2.grid(True)

    # Plot 3: Precision-Recall Curve
    ax3 = fig.add_subplot(3, 3, 3*row + 3)
    ax3.plot(recall, precision, label="PR Curve")
    ax3.set_title(f"{set_name} - Precision-Recall Curve")
    ax3.set_xlabel("Recall")
    ax3.set_ylabel("Precision")
    ax3.legend()
    ax3.grid(True)



fig = plt.figure(figsize=(18, 12))
plot_graphs_only(log_reg_balanced, X_train, y_train, "Training (Oversampled)", 0, fig)
plot_graphs_only(log_reg_balanced, X_val, y_val, "Validation", 1, fig)
plot_graphs_only(log_reg_balanced, X_test, y_test, "Test", 2, fig)
plt.tight_layout()
plt.show()


# In[40]:


def evaluate_lgwt(y_true, y_pred, X_data, dataset_name):
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)

    fpr, tpr, _ = roc_curve(y_true, log_reg_balanced.predict_proba(X_data)[:, 1])
    eer = fpr[np.nanargmin(np.abs((1 - tpr) - fpr))]

    print(f"{dataset_name} Results:")
    print(f"Accuracy: {acc:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Recall: {rec:.4f}")
    print(f"Confusion Matrix:\n{cm}")
    print(f"Equal Error Rate (EER): {eer:.4f}\n")

# Step 5: Run evaluations
evaluate_lgwt(y_train, y_train_pred, X_train, "Training")
evaluate_lgwt(y_val, y_val_pred, X_val, "Validation")
evaluate_lgwt(y_test, y_test_pred, X_test, "Test (Evaluation)")


# In[41]:


import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

def plot_confusion_matrices(y_tr, y_tr_pred, y_val, y_val_pred, y_test, y_test_pred):
    titles = ["Training Set", "Validation Set", "Test Set"]
    datasets = [(y_tr, y_tr_pred), (y_val, y_val_pred), (y_test, y_test_pred)]

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    for ax, (y_true, y_pred), title in zip(axes, datasets, titles):
        cm = confusion_matrix(y_true, y_pred)
        sns.heatmap(cm, annot=True, fmt="d", cmap="coolwarm", cbar=False, ax=ax)
        ax.set_title(f"Confusion Matrix - {title}")
        ax.set_xlabel("Predicted")
        ax.set_ylabel("Actual")

    plt.tight_layout()
    plt.show()
plot_confusion_matrices(y_train, y_train_pred, y_val, y_val_pred, y_test, y_test_pred)


# In[ ]:






## Random Forest (Abbas)

### Random Forest -  Random Search

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from scipy.stats import randint
import matplotlib.pyplot as plt
import time
import seaborn as sns

In [ ]:
train_df = pd.read_csv('train.csv')
dev_df = pd.read_csv('dev.csv')
eval_df = pd.read_csv('eval.csv')

In [ ]:
X_train = train_df.iloc[:, 2:].values
y_train = train_df["label"].values

X_dev = dev_df.iloc[:, 2:].values
y_dev = dev_df["label"].values

X_test = eval_df.iloc[:, 2:].values
y_test = eval_df["label"].values

In [ ]:
param_dist = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt'],
    'bootstrap': [True],
    'class_weight': ['balanced']
}

rf = RandomForestClassifier(random_state=42)


random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=10,
    verbose=1,
    cv=2,
    random_state=42,
    n_jobs=-1
)


random_search.fit(X_train, y_train)

In [ ]:
best_rf = random_search.best_estimator_
print("Best Parameters:", random_search.best_params_)

y_dev_pred = best_rf.predict(X_dev)
y_test_pred = best_rf.predict(X_test)

In [ ]:
def plot_cm(y_true, y_pred, title, cmap='viridis'):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(4, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap=cmap,
                xticklabels=['Class 0', 'Class 1'],
                yticklabels=['Class 0', 'Class 1'])
    plt.title(f'Confusion Matrix ({title})')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.tight_layout()
    plt.show()

In [ ]:
print("\nValidation (Dev) Results:")
print("Accuracy:", accuracy_score(y_dev, y_dev_pred))
print("Classification Report:\n", classification_report(y_dev, y_dev_pred, zero_division=0))
plot_cm(y_dev, y_dev_pred, "Dev Set", cmap='mako')

print("\nTest (Eval) Results:")
print("Accuracy:", accuracy_score(y_test, y_test_pred))
print("Classification Report:\n", classification_report(y_test, y_test_pred, zero_division=0))
plot_cm(y_test, y_test_pred, "Test Set", cmap='rocket')

### Random Forest Original

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from scipy.stats import randint
import matplotlib.pyplot as plt
import time
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
# Load the CSVs
train_df = pd.read_csv('train.csv')
train_df = pd.read_csv('train.csv')
eval_df = pd.read_csv('eval.csv')

In [ ]:
X_train = train_df.iloc[:, 2:].values
y_train = train_df["label"].values

X_train = train_df.iloc[:, 2:].values
y_train = train_df["label"].values

X_test = eval_df.iloc[:, 2:].values
y_test = eval_df["label"].values


rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)



In [ ]:
def evaluate_model(name, model, X, y_true):
    print(f"=== {name} Set Evaluation ===")
    y_pred = model.predict(X)
    y_proba = model.predict_proba(X)[:, 1]


    print(confusion_matrix(y_true, y_pred))
    print(classification_report(y_true, y_pred))
    print("Accuracy :", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred, average='weighted'))
    print("Recall   :", recall_score(y_true, y_pred, average='weighted'))
    print("F1 Score :", f1_score(y_true, y_pred, average='weighted'))

    fpr, tpr, thresholds = roc_curve(y_true, y_proba)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    print("Equal Error Rate (EER):", round(eer, 4))
    print("\n")

evaluate_model("Train", rf_model, X_train, y_train)
evaluate_model("Dev", rf_model, X_dev, y_dev)
evaluate_model("Eval", rf_model, X_test, y_test)

### Random Forest - Sampled

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from scipy.stats import randint
import matplotlib.pyplot as plt
import time
import seaborn as sns

In [ ]:
train_df = pd.read_csv('train.csv')
dev_df = pd.read_csv('dev.csv')
eval_df = pd.read_csv('eval.csv')

X_train = train_df.iloc[:, 2:].values
y_train = train_df["label"].values

X_dev = dev_df.iloc[:, 2:].values
y_dev = dev_df["label"].values

X_test = eval_df.iloc[:, 2:].values
y_test = eval_df["label"].values

In [ ]:
#through randomized search method i have obtianed the best parameters and i have got the following.i am performing smote and undersampling on them.
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier

smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X_train, y_train)

model = RandomForestClassifier(
    n_estimators=100,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='sqrt',
    max_depth=None,
    class_weight='balanced',
    bootstrap=True,
    random_state=42
)
model.fit(X_smote, y_smote)

def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_idx = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_idx] + fnr[eer_idx]) / 2
    return eer

def evaluate(X, y_true, name):
    y_pred = model.predict(X)
    y_proba = model.predict_proba(X)[:, 1]

    acc = accuracy_score(y_true, y_pred)
    err = 1 - acc
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    eer = compute_eer(y_true, y_proba)

    print(f"\n{name} Results:")
    print(f"Accuracy:           {acc:.4f}")
    print(f"Error Rate:         {err:.4f}")
    print(f"Weighted Precision: {precision:.4f}")
    print(f"Weighted Recall:    {recall:.4f}")
    print(f"Weighted F1 Score:  {f1:.4f}")
    print(f"Equal Error Rate:   {eer:.4f}")
    print("Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))

evaluate(X_smote, y_smote, "Train")
evaluate(X_dev, y_dev, "Dev")
evaluate(X_test, y_test, "Test")


In [ ]:
from sklearn.metrics import classification_report

def f1_per_class(X, y_true, name):
    y_pred = model.predict(X)
    report = classification_report(y_true, y_pred, output_dict=True, zero_division=0)

    f1_0 = report['0']['f1-score']
    f1_1 = report['1']['f1-score']

    print(f"\n{name} Set F1 Scores:")
    print(f"F1 Score (Label 0): {f1_0:.4f}")
    print(f"F1 Score (Label 1): {f1_1:.4f}")

f1_per_class(X_smote, y_smote, "Train")
f1_per_class(X_dev, y_dev, "Dev")
f1_per_class(X_test, y_test, "Test")


In [ ]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=42)
X_rus, y_rus = rus.fit_resample(X_train, y_train)

model = RandomForestClassifier(
    n_estimators=100,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='sqrt',
    max_depth=None,
    class_weight='balanced',
    bootstrap=True,
    random_state=42
)
model.fit(X_rus, y_rus)

def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_idx = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_idx] + fnr[eer_idx]) / 2
    return eer

def evaluate(X, y_true, name):
    y_pred = model.predict(X)
    y_proba = model.predict_proba(X)[:, 1]

    acc = accuracy_score(y_true, y_pred)
    err = 1 - acc
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    eer = compute_eer(y_true, y_proba)

    print(f"\n{name} Results:")
    print(f"Accuracy:           {acc:.4f}")
    print(f"Error Rate:         {err:.4f}")
    print(f"Weighted Precision: {precision:.4f}")
    print(f"Weighted Recall:    {recall:.4f}")
    print(f"Weighted F1 Score:  {f1:.4f}")
    print(f"Equal Error Rate:   {eer:.4f}")
    print("Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))

evaluate(X_rus, y_rus, "Train")
evaluate(X_dev, y_dev, "Dev")
evaluate(X_test, y_test, "Test")


In [ ]:
from sklearn.metrics import classification_report

def f1_per_class(X, y_true, name):
    y_pred = model.predict(X)
    report = classification_report(y_true, y_pred, output_dict=True, zero_division=0)

    f1_0 = report['0']['f1-score']
    f1_1 = report['1']['f1-score']

    print(f"\n{name} Set F1 Scores:")
    print(f"F1 Score (Label 0): {f1_0:.4f}")
    print(f"F1 Score (Label 1): {f1_1:.4f}")

f1_per_class(X_rus, y_rus, "Train")
f1_per_class(X_dev, y_dev, "Dev")
f1_per_class(X_test, y_test, "Test")


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_dev_scaled = scaler.transform(X_dev)
X_test_scaled = scaler.transform(X_test)

pca = PCA(n_components=100, random_state=42)
X_train_pca = pca.fit_transform(X_train_scaled)
X_dev_pca = pca.transform(X_dev_scaled)
X_test_pca = pca.transform(X_test_scaled)

model = RandomForestClassifier(
    n_estimators=100,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='sqrt',
    max_depth=None,
    class_weight='balanced',
    bootstrap=True,
    random_state=42
)
model.fit(X_train_pca, y_train)

def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_idx = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_idx] + fnr[eer_idx]) / 2
    return eer

def evaluate(X, y_true, name):
    y_pred = model.predict(X)
    y_proba = model.predict_proba(X)[:, 1]

    acc = accuracy_score(y_true, y_pred)
    err = 1 - acc
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    eer = compute_eer(y_true, y_proba)

    print(f"\n{name} Results (with PCA):")
    print(f"Accuracy:           {acc:.4f}")
    print(f"Error Rate:         {err:.4f}")
    print(f"Weighted Precision: {precision:.4f}")
    print(f"Weighted Recall:    {recall:.4f}")
    print(f"Weighted F1 Score:  {f1:.4f}")
    print(f"Equal Error Rate:   {eer:.4f}")
    print("Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))

evaluate(X_train_pca, y_train, "Train")
evaluate(X_dev_pca, y_dev, "Dev")
evaluate(X_test_pca, y_test, "Test")


In [ ]:
from sklearn.metrics import classification_report

def f1_per_class(X, y_true, name):
    y_pred = model.predict(X)
    report = classification_report(y_true, y_pred, output_dict=True, zero_division=0)

    f1_0 = report['0']['f1-score']
    f1_1 = report['1']['f1-score']

    print(f"\n{name} Set F1 Scores:")
    print(f"F1 Score (Label 0): {f1_0:.4f}")
    print(f"F1 Score (Label 1): {f1_1:.4f}")

f1_per_class(X_train_pca, y_train, "Train")
f1_per_class(X_dev_pca, y_dev, "Dev")
f1_per_class(X_test_pca, y_test, "Test")


## Naive Bayes (Abbas)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pandas as pd
from sklearn.metrics import roc_curve
import numpy as np
from sklearn.metrics import (accuracy_score, f1_score, precision_score, recall_score)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
train_df = pd.read_csv('train.csv')
dev_df = pd.read_csv('dev.csv')
eval_df = pd.read_csv('eval.csv')

X_train = train_df.iloc[:, 2:].values
y_train = train_df["label"].values
X_dev = dev_df.iloc[:, 2:].values
y_dev = dev_df["label"].values
X_test = eval_df.iloc[:, 2:].values
y_test = eval_df["label"].values

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_dev_scaled = scaler.transform(X_dev)
X_test_scaled = scaler.transform(X_test)

#training
nb_model = GaussianNB()
nb_model.fit(X_train_scaled, y_train)

In [ ]:
def evaluate_model(name, model, X, y_true):
    print(f"=== {name} Set Evaluation ===")
    y_pred = model.predict(X)
    y_proba = model.predict_proba(X)[:, 1]

    print(confusion_matrix(y_true, y_pred))
    print(classification_report(y_true, y_pred))
    print("Accuracy :", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred, average='weighted'))
    print("Recall   :", recall_score(y_true, y_pred, average='weighted'))
    print("F1 Score :", f1_score(y_true, y_pred, average='weighted'))

    # EER
    fpr, tpr, thresholds = roc_curve(y_true, y_proba)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    print("Equal Error Rate (EER):", round(eer, 4))
    print("\n")

# Evaluations
evaluate_model("Train", nb_model, X_train_scaled, y_train)
evaluate_model("Dev", nb_model, X_dev_scaled, y_dev)
evaluate_model("Eval", nb_model, X_test_scaled, y_test)

In [ ]:
y_dev_pred = nb_model.predict(X_dev_scaled)
y_test_pred = nb_model.predict(X_test_scaled)
plt.figure(figsize=(6, 5))
sns.heatmap(confusion_matrix(y_dev, y_dev_pred), annot=True, fmt="d", cmap="coolwarm", cbar=False)
plt.title("Confusion Matrix (Validation Set)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

plt.figure(figsize=(6, 5))
sns.heatmap(confusion_matrix(y_test, y_test_pred), annot=True, fmt="d", cmap="coolwarm", cbar=False)
plt.title("Confusion Matrix (Test Set)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

In [ ]:
pca = PCA(n_components=100)
X_train_pca = pca.fit_transform(X_train_scaled)
X_dev_pca = pca.transform(X_dev_scaled)
X_test_pca = pca.transform(X_test_scaled)

# Training on pca data
nb_model_1 = GaussianNB()
nb_model_1.fit(X_train_pca, y_train)

In [ ]:
def evaluate_model(name, model, X, y_true):
    print(f"=== {name} Set Evaluation ===")
    y_pred = model.predict(X)
    y_proba = model.predict_proba(X)[:, 1]

    print(confusion_matrix(y_true, y_pred))
    print(classification_report(y_true, y_pred))
    print("Accuracy :", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred, average='weighted'))
    print("Recall   :", recall_score(y_true, y_pred, average='weighted'))
    print("F1 Score :", f1_score(y_true, y_pred, average='weighted'))

    # EER
    fpr, tpr, thresholds = roc_curve(y_true, y_proba)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    print("Equal Error Rate (EER):", round(eer, 4))
    print("\n")

# Evaluations
evaluate_model("Train", nb_model_1, X_train_pca, y_train)
evaluate_model("Dev", nb_model_1, X_dev_pca, y_dev)
evaluate_model("Eval", nb_model_1, X_test_pca, y_test)

In [ ]:
y_dev_pred = nb_model_1.predict(X_dev_pca)
y_test_pred = nb_model_1.predict(X_test_pca)
plt.figure(figsize=(6, 5))
sns.heatmap(confusion_matrix(y_dev, y_dev_pred), annot=True, fmt="d", cmap="coolwarm", cbar=False)
plt.title("Confusion Matrix (Validation Set)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

plt.figure(figsize=(6, 5))
sns.heatmap(confusion_matrix(y_test, y_test_pred), annot=True, fmt="d", cmap="coolwarm", cbar=False)
plt.title("Confusion Matrix (Test Set)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

In [ ]:
#Applying SMOTE
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

#Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_balanced)
X_dev_scaled = scaler.transform(X_dev)
X_test_scaled = scaler.transform(X_test)

#PCA
pca = PCA(n_components=100)
X_train_pca = pca.fit_transform(X_train_scaled)
X_dev_pca = pca.transform(X_dev_scaled)
X_test_pca = pca.transform(X_test_scaled)

nb_model_2 = GaussianNB()
nb_model_2.fit(X_train_pca, y_train_balanced)

from sklearn.metrics import (
    confusion_matrix, classification_report, accuracy_score,
    precision_score, recall_score, f1_score, roc_curve
)
import numpy as np

def evaluate_model(name, model, X, y_true):
    print(f"=== {name} Set Evaluation ===")
    y_pred = model.predict(X)
    y_proba = model.predict_proba(X)[:, 1]

    # Metrics
    print(confusion_matrix(y_true, y_pred))
    print(classification_report(y_true, y_pred))
    print("Accuracy :", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred, average='weighted'))
    print("Recall   :", recall_score(y_true, y_pred, average='weighted'))
    print("F1 Score :", f1_score(y_true, y_pred, average='weighted'))

    # EER
    fpr, tpr, thresholds = roc_curve(y_true, y_proba)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fnr - fpr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    print("Equal Error Rate (EER):", round(eer, 4))
    print("\n")

#Evaluations
evaluate_model("Train", nb_model_2, X_train_pca, y_train_balanced)
evaluate_model("Dev", nb_model_2, X_dev_pca, y_dev)
evaluate_model("Eval", nb_model_2, X_test_pca, y_test)

In [ ]:
y_dev_pred = nb_model_2.predict(X_dev_pca)
y_test_pred = nb_model_2.predict(X_test_pca)

plt.figure(figsize=(6, 5))
sns.heatmap(confusion_matrix(y_dev, y_dev_pred), annot=True, fmt="d", cmap="coolwarm", cbar=False)
plt.title("Confusion Matrix (Validation Set)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

plt.figure(figsize=(6, 5))
sns.heatmap(confusion_matrix(y_test, y_test_pred), annot=True, fmt="d", cmap="coolwarm", cbar=False)
plt.title("Confusion Matrix (Test Set)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

In [ ]:
#Applying Random Undersampling
undersampler = RandomUnderSampler(random_state=42)
X_train_balanced, y_train_balanced = undersampler.fit_resample(X_train, y_train)

#Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_balanced)
X_dev_scaled = scaler.transform(X_dev)
X_test_scaled = scaler.transform(X_test)

# PCA
pca = PCA(n_components=100)
X_train_pca = pca.fit_transform(X_train_scaled)
X_dev_pca = pca.transform(X_dev_scaled)
X_test_pca = pca.transform(X_test_scaled)


nb_model_3 = GaussianNB()
nb_model_3.fit(X_train_pca, y_train_balanced)

from sklearn.metrics import (
    confusion_matrix, classification_report, accuracy_score,
    precision_score, recall_score, f1_score, roc_curve
)
import numpy as np

#Evaluation
def evaluate_model(name, model, X, y_true):
    print(f"\n=== {name} Set Evaluation ===")

    y_pred = model.predict(X)
    y_proba = model.predict_proba(X)[:, 1]

    print(confusion_matrix(y_true, y_pred))
    print(classification_report(y_true, y_pred))

    print("Accuracy :", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred, average='weighted'))
    print("Recall   :", recall_score(y_true, y_pred, average='weighted'))
    print("F1 Score :", f1_score(y_true, y_pred, average='weighted'))

    #EER
    fpr, tpr, thresholds = roc_curve(y_true, y_proba)
    fnr = 1 - tpr
    eer_index = np.nanargmin(np.abs(fpr - fnr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    print("Equal Error Rate (EER):", round(eer, 4))

#Evaluation
evaluate_model("Train", nb_model_3, X_train_pca, y_train_balanced)
evaluate_model("Dev", nb_model_3, X_dev_pca, y_dev)
evaluate_model("Eval", nb_model_3, X_test_pca, y_test)


In [ ]:
y_dev_pred = nb_model_3.predict(X_dev_pca)
y_test_pred = nb_model_3.predict(X_test_pca)

plt.figure(figsize=(6, 5))
sns.heatmap(confusion_matrix(y_dev, y_dev_pred), annot=True, fmt="d", cmap="coolwarm", cbar=False)
plt.title("Confusion Matrix (Validation Set)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

plt.figure(figsize=(6, 5))
sns.heatmap(confusion_matrix(y_test, y_test_pred), annot=True, fmt="d", cmap="coolwarm", cbar=False)
plt.title("Confusion Matrix (Test Set)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

## Decision Tree (Abbas)

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import (confusion_matrix, classification_report, accuracy_score,precision_score, recall_score, f1_score, roc_curve)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
train_df = pd.read_csv('train.csv')
dev_df = pd.read_csv('dev.csv')
eval_df = pd.read_csv('eval.csv')

X_train = train_df.iloc[:, 2:].values
y_train = train_df["label"].values
X_dev = dev_df.iloc[:, 2:].values
y_dev = dev_df["label"].values
X_test = eval_df.iloc[:, 2:].values
y_test = eval_df["label"].values

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_dev_scaled = scaler.transform(X_dev)
X_test_scaled = scaler.transform(X_test)

clf = DecisionTreeClassifier(
    criterion='gini',
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='sqrt',
    class_weight='balanced',
    random_state=42
)
clf.fit(X_train_scaled, y_train)

# Evaluation
def evaluate_model(name, model, X, y_true):
    print(f"\n=== {name} Set Evaluation ===")

    y_pred = model.predict(X)

    # EER
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X)[:, 1]
        fpr, tpr, _ = roc_curve(y_true, y_proba)
        fnr = 1 - tpr
        eer_index = np.nanargmin(np.abs(fpr - fnr))
        eer = (fpr[eer_index] + fnr[eer_index]) / 2
    else:
        eer = None

    print(confusion_matrix(y_true, y_pred))
    print(classification_report(y_true, y_pred))
    print("Accuracy :", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred, average='weighted'))
    print("Recall   :", recall_score(y_true, y_pred, average='weighted'))
    print("F1 Score :", f1_score(y_true, y_pred, average='weighted'))

    if eer is not None:
        print("Equal Error Rate (EER):", round(eer, 4))
    else:
        print("Equal Error Rate (EER): Not available")

evaluate_model("Train", clf, X_train_scaled, y_train)
evaluate_model("Dev", clf, X_dev_scaled, y_dev)
evaluate_model("Eval", clf, X_test_scaled, y_test)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_dev_scaled = scaler.transform(X_dev)
X_test_scaled = scaler.transform(X_test)

clf = DecisionTreeClassifier(
    criterion='gini',
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='sqrt',
    class_weight='balanced',
    random_state=42
)
clf.fit(X_train_scaled, y_train)

# Evaluation
def evaluate_model(name, model, X, y_true):
    print(f"\n=== {name} Set Evaluation ===")

    y_pred = model.predict(X)

    # EER
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X)[:, 1]
        fpr, tpr, _ = roc_curve(y_true, y_proba)
        fnr = 1 - tpr
        eer_index = np.nanargmin(np.abs(fpr - fnr))
        eer = (fpr[eer_index] + fnr[eer_index]) / 2
    else:
        eer = None

    print(confusion_matrix(y_true, y_pred))
    print(classification_report(y_true, y_pred))
    print("Accuracy :", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred))
    print("Recall   :", recall_score(y_true, y_pred))
    print("F1 Score :", f1_score(y_true, y_pred))

    if eer is not None:
        print("Equal Error Rate (EER):", round(eer, 4))
    else:
        print("Equal Error Rate (EER): Not available")

evaluate_model("Train", clf, X_train_scaled, y_train)
evaluate_model("Dev", clf, X_dev_scaled, y_dev)
evaluate_model("Eval", clf, X_test_scaled, y_test)

In [ ]:
y_dev_pred = clf.predict(X_dev_scaled)
y_test_pred = clf.predict(X_test_scaled)

plt.figure(figsize=(6, 5))
sns.heatmap(confusion_matrix(y_dev, y_dev_pred), annot=True, fmt="d", cmap="coolwarm", cbar=False)
plt.title("Confusion Matrix (Validation Set)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

plt.figure(figsize=(6, 5))
sns.heatmap(confusion_matrix(y_test, y_test_pred), annot=True, fmt="d", cmap="coolwarm", cbar=False)
plt.title("Confusion Matrix (Test Set)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

In [ ]:
#PCA
pca = PCA(n_components=100)
X_train_pca = pca.fit_transform(X_train_scaled)
X_dev_pca = pca.transform(X_dev_scaled)
X_test_pca = pca.transform(X_test_scaled)

clf_1 = DecisionTreeClassifier(
    criterion='gini',
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='sqrt',
    class_weight='balanced',
    random_state=42
)
clf_1.fit(X_train_pca, y_train)
from sklearn.metrics import (
    confusion_matrix, classification_report, accuracy_score,
    precision_score, recall_score, f1_score, roc_curve
)
import numpy as np

# === Evaluation Function ===
def evaluate_model(name, model, X, y_true):
    print(f"\n=== {name} Set Evaluation ===")

    y_pred = model.predict(X)

    # Predict probabilities only if supported
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X)[:, 1]  # Binary class 1 prob
        fpr, tpr, thresholds = roc_curve(y_true, y_proba)
        fnr = 1 - tpr
        eer_index = np.nanargmin(np.abs(fpr - fnr))
        eer = (fpr[eer_index] + fnr[eer_index]) / 2
    else:
        eer = None

    # Print Metrics
    print(confusion_matrix(y_true, y_pred))
    print(classification_report(y_true, y_pred))
    print("Accuracy :", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred, average='weighted'))
    print("Recall   :", recall_score(y_true, y_pred, average='weighted'))
    print("F1 Score :", f1_score(y_true, y_pred, average='weighted'))

    if eer is not None:
        print("Equal Error Rate (EER):", round(eer, 4))
    else:
        print("Equal Error Rate (EER): Not available")

# === Run for Train, Dev, Eval ===
evaluate_model("Train", clf_1, X_train_pca, y_train)
evaluate_model("Dev", clf_1, X_dev_pca, y_dev)
evaluate_model("Eval", clf_1, X_test_pca, y_test)


In [ ]:
#PCA
pca = PCA(n_components=100)
X_train_pca = pca.fit_transform(X_train_scaled)
X_dev_pca = pca.transform(X_dev_scaled)
X_test_pca = pca.transform(X_test_scaled)

clf_1 = DecisionTreeClassifier(
    criterion='gini',
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='sqrt',
    class_weight='balanced',
    random_state=42
)
clf_1.fit(X_train_pca, y_train)
from sklearn.metrics import (
    confusion_matrix, classification_report, accuracy_score,
    precision_score, recall_score, f1_score, roc_curve
)
import numpy as np

# === Evaluation Function ===
def evaluate_model(name, model, X, y_true):
    print(f"\n=== {name} Set Evaluation ===")

    y_pred = model.predict(X)

    # Predict probabilities only if supported
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X)[:, 1]  # Binary class 1 prob
        fpr, tpr, thresholds = roc_curve(y_true, y_proba)
        fnr = 1 - tpr
        eer_index = np.nanargmin(np.abs(fpr - fnr))
        eer = (fpr[eer_index] + fnr[eer_index]) / 2
    else:
        eer = None

    # Print Metrics
    print(confusion_matrix(y_true, y_pred))
    print(classification_report(y_true, y_pred))
    print("Accuracy :", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred))
    print("Recall   :", recall_score(y_true, y_pred))
    print("F1 Score :", f1_score(y_true, y_pred))

    if eer is not None:
        print("Equal Error Rate (EER):", round(eer, 4))
    else:
        print("Equal Error Rate (EER): Not available")

# === Run for Train, Dev, Eval ===
evaluate_model("Train", clf_1, X_train_pca, y_train)
evaluate_model("Dev", clf_1, X_dev_pca, y_dev)
evaluate_model("Eval", clf_1, X_test_pca, y_test)


In [ ]:
y_dev_pred = clf_1.predict(X_dev_pca)
y_test_pred = clf_1.predict(X_test_pca)

plt.figure(figsize=(6, 5))
sns.heatmap(confusion_matrix(y_dev, y_dev_pred), annot=True, fmt="d", cmap="coolwarm", cbar=False)
plt.title("Confusion Matrix (Validation Set)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

plt.figure(figsize=(6, 5))
sns.heatmap(confusion_matrix(y_test, y_test_pred), annot=True, fmt="d", cmap="coolwarm", cbar=False)
plt.title("Confusion Matrix (Test Set)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

In [ ]:
#smote & PCA100
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

pca = PCA(n_components=100)
X_train_resampled_scaled = scaler.fit_transform(X_train_resampled)
X_train_resampled_pca = pca.fit_transform(X_train_resampled_scaled)
X_dev_pca = pca.transform(X_dev_scaled)
X_test_pca = pca.transform(X_test_scaled)

clf_smote = DecisionTreeClassifier(
    criterion='gini',
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='sqrt',
    class_weight='balanced',
    random_state=42
)
clf_smote.fit(X_train_resampled_pca, y_train_resampled)
def evaluate_model(name, model, X, y_true):
    print(f"\n=== {name} Set Evaluation ===")
    y_pred = model.predict(X)

    # EER Calculation (binary only)
    eer = None
    if hasattr(model, "predict_proba") and len(np.unique(y_true)) == 2:
        y_scores = model.predict_proba(X)[:, 1]
        fpr, tpr, _ = roc_curve(y_true, y_scores)
        fnr = 1 - tpr
        eer_index = np.nanargmin(np.abs(fpr - fnr))
        eer = (fpr[eer_index] + fnr[eer_index]) / 2

    # Metrics Output
    print(confusion_matrix(y_true, y_pred))
    print(classification_report(y_true, y_pred))
    print("Accuracy :", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred, average='weighted'))
    print("Recall   :", recall_score(y_true, y_pred, average='weighted'))
    print("F1 Score :", f1_score(y_true, y_pred, average='weighted'))
    print("Equal Error Rate (EER):", f"{eer:.4f}" if eer is not None else "Not available")

# === 6. Run Evaluation on Train, Dev, and Test Sets ===
evaluate_model("Train", clf_smote, X_train_resampled_pca, y_train_resampled)
evaluate_model("Dev", clf_smote, X_dev_pca, y_dev)
evaluate_model("Eval", clf_smote, X_test_pca, y_test)

In [ ]:
y_dev_pred = clf_smote.predict(X_dev_pca)
y_test_pred = clf_smote.predict(X_test_pca)

plt.figure(figsize=(6, 5))
sns.heatmap(confusion_matrix(y_dev, y_dev_pred), annot=True, fmt="d", cmap="coolwarm", cbar=False)
plt.title("Confusion Matrix (Validation Set)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

plt.figure(figsize=(6, 5))
sns.heatmap(confusion_matrix(y_test, y_test_pred), annot=True, fmt="d", cmap="coolwarm", cbar=False)
plt.title("Confusion Matrix (Test Set)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

In [ ]:
#smote & PCA100 &metrics not weighted
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

pca = PCA(n_components=100)
X_train_resampled_scaled = scaler.fit_transform(X_train_resampled)
X_train_resampled_pca = pca.fit_transform(X_train_resampled_scaled)
X_dev_pca = pca.transform(X_dev_scaled)
X_test_pca = pca.transform(X_test_scaled)

clf_smote = DecisionTreeClassifier(
    criterion='gini',
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='sqrt',
    class_weight='balanced',
    random_state=42
)
clf_smote.fit(X_train_resampled_pca, y_train_resampled)
def evaluate_model(name, model, X, y_true):
    print(f"\n=== {name} Set Evaluation ===")
    y_pred = model.predict(X)

    # EER
    eer = None
    if hasattr(model, "predict_proba") and len(np.unique(y_true)) == 2:
        y_scores = model.predict_proba(X)[:, 1]
        fpr, tpr, _ = roc_curve(y_true, y_scores)
        fnr = 1 - tpr
        eer_index = np.nanargmin(np.abs(fpr - fnr))
        eer = (fpr[eer_index] + fnr[eer_index]) / 2

    print(confusion_matrix(y_true, y_pred))
    print(classification_report(y_true, y_pred))
    print("Accuracy :", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred))
    print("Recall   :", recall_score(y_true, y_pred))
    print("F1 Score :", f1_score(y_true, y_pred))
    print("Equal Error Rate (EER):", f"{eer:.4f}" if eer is not None else "Not available")

evaluate_model("Train", clf_smote, X_train_resampled_pca, y_train_resampled)
evaluate_model("Dev", clf_smote, X_dev_pca, y_dev)
evaluate_model("Eval", clf_smote, X_test_pca, y_test)

In [ ]:
#Undersampling before PCA
undersampler = RandomUnderSampler(random_state=42)
X_train_under, y_train_under = undersampler.fit_resample(X_train, y_train)

X_train_under_scaled = scaler.fit_transform(X_train_under)
X_train_under_pca = pca.fit_transform(X_train_under_scaled)
X_dev_pca = pca.transform(X_dev_scaled)
X_test_pca = pca.transform(X_test_scaled)

clf_under = DecisionTreeClassifier(
    criterion='gini',
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='sqrt',
    class_weight='balanced',
    random_state=42
)
clf_under.fit(X_train_under_pca, y_train_under)

def evaluate_model(name, model, X, y_true):
    print(f"\n=== {name} Set Evaluation ===")
    y_pred = model.predict(X)

    # EER
    eer = None
    if hasattr(model, "predict_proba") and len(np.unique(y_true)) == 2:
        y_scores = model.predict_proba(X)[:, 1]
        fpr, tpr, _ = roc_curve(y_true, y_scores)
        fnr = 1 - tpr
        eer_index = np.nanargmin(np.abs(fpr - fnr))
        eer = (fpr[eer_index] + fnr[eer_index]) / 2

    print(confusion_matrix(y_true, y_pred))
    print(classification_report(y_true, y_pred))
    print("Accuracy :", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred))
    print("Recall   :", recall_score(y_true, y_pred))
    print("F1 Score :", f1_score(y_true, y_pred))
    print("Equal Error Rate (EER):", f"{eer:.4f}" if eer is not None else "Not available")

evaluate_model("Train", clf_under, X_train_under_pca, y_train_under)
evaluate_model("Dev", clf_under, X_dev_pca, y_dev)
evaluate_model("Eval", clf_under, X_test_pca, y_test)



In [ ]:
#Undersampling before PCA
undersampler = RandomUnderSampler(random_state=42)
X_train_under, y_train_under = undersampler.fit_resample(X_train, y_train)

X_train_under_scaled = scaler.fit_transform(X_train_under)
X_train_under_pca = pca.fit_transform(X_train_under_scaled)
X_dev_pca = pca.transform(X_dev_scaled)
X_test_pca = pca.transform(X_test_scaled)

clf_under = DecisionTreeClassifier(
    criterion='gini',
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='sqrt',
    class_weight='balanced',
    random_state=42
)
clf_under.fit(X_train_under_pca, y_train_under)

def evaluate_model(name, model, X, y_true):
    print(f"\n=== {name} Set Evaluation ===")
    y_pred = model.predict(X)

    # EER
    eer = None
    if hasattr(model, "predict_proba") and len(np.unique(y_true)) == 2:
        y_scores = model.predict_proba(X)[:, 1]
        fpr, tpr, _ = roc_curve(y_true, y_scores)
        fnr = 1 - tpr
        eer_index = np.nanargmin(np.abs(fpr - fnr))
        eer = (fpr[eer_index] + fnr[eer_index]) / 2

    print(confusion_matrix(y_true, y_pred))
    print(classification_report(y_true, y_pred))
    print("Accuracy :", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred,average='weighted'))
    print("Recall   :", recall_score(y_true, y_pred,average='weighted'))
    print("F1 Score :", f1_score(y_true, y_pred,average='weighted'))
    print("Equal Error Rate (EER):", f"{eer:.4f}" if eer is not None else "Not available")

evaluate_model("Train", clf_under, X_train_under_pca, y_train_under)
evaluate_model("Dev", clf_under, X_dev_pca, y_dev)
evaluate_model("Eval", clf_under, X_test_pca, y_test)



In [ ]:
y_dev_pred = clf_under.predict(X_dev_pca)
y_test_pred = clf_under.predict(X_test_pca)

plt.figure(figsize=(6, 5))
sns.heatmap(confusion_matrix(y_dev, y_dev_pred), annot=True, fmt="d", cmap="coolwarm", cbar=False)
plt.title("Confusion Matrix (Validation Set)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

plt.figure(figsize=(6, 5))
sns.heatmap(confusion_matrix(y_test, y_test_pred), annot=True, fmt="d", cmap="coolwarm", cbar=False)
plt.title("Confusion Matrix (Test Set)")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()